# Bag of Words-Modelle für Politiker, nach Bundesländern gegliedert

In [1]:
# loading libs
import pandas as pd
import numpy as np
import pprint as pp
import Stemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import Stemmer

## Datensätze Politiker nach Ländern

In [2]:
polBawü = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_baden-württemberg.csv")
polBay = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_bayern.csv")
polBerlin = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_berlin.csv")
polBrandenburg = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_brandenburg.csv")
polBremen = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_bremen.csv")
polHamburg = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_hamburg.csv")
polHessen = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_hessen.csv")
polMeckPomm = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_meckPomm.csv")
polNds = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_niedersachsen.csv")
polNrw = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_nrw.csv")
polRlp = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_rlp.csv")
polSaar = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_saarland.csv")
polSachsen = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_sachsen.csv")
polSachsenAnhalt = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_sachsen-anhalt.csv")
polSchleswig = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_schleswig-holstein.csv") 
polThüringen = pd.read_csv("../../methods/representative-speech/regionalisierung/politiker-regionalisiert/politiker_thüringen.csv")

## ÜBERGREIFENDE THEMEN FÜR ALLE LÄNDER

In [3]:
themenliste = {
    'protesteIran' : ['iran', 'iraner', 'iranerin', 'iranerinnen', 'iranisch', 'iranische', 'iranischen',
                      'mullah', 'mullahs', 'mullah-regime', 'theokratie', 'scharia', 'jin', 'jiyan', 'azadi',
                      'jina', 'mahsa', 'amini', 'mahsaamini', 'kopftuch', 'hijab', 'hidschab', 'jinjiyanazadi',
                      'frauenlebenfreiheit', 'womenlifefreedom', 'internetblockade', 'freeiran',
                      'iranprotests', 'iranprotests2022', 'iranrevolution', 'iranrevolution2022',
                      'sittenwächter', 'sittenpolizei', 'sittenpolizist', 'sittenpolizisten',
                      'kopftuchprotest', 'kopftuchproteste', 'hedschab', 'kopftücher', 'ajatollah', 'ayatollah',
                      'chamenei', 'khamenei'],
    
    'klima' : ['klima', 'klimaschutz', 'klimakrise', 'energiewende', 'erwärmung', 'erderwärmung',
               'klimaschutzgesetz', 'solar', 'solarenergie', 'photovoltaik', 'wasserkraft', 'kohleausstieg',
               'erneuerbare', 'erneuerbaren', 'wind', 'windkraft', 'windkraftwerk', 'lützerath', 'ausbau',
               'climateaction', 'fridaysforfuture', 'fff', 'greta', 'thunberg', 'klimakanzler', 'nachhaltig',
               'nachhaltige', 'nachhaltiger', 'nachhaltigkeit', 'letztegeneration',' klimakleber',
               'klimaprotest', 'klimaproteste', 'co2', 'sitzblockade', 'sitzblockaden', 'ökosystem',
               'ökosysteme', 'kipppunkt', 'kipppunkte', 'solartechnik', 'photovoltaikanlage',
               'photovoltaikanlagen', 'photovoltaik-anlage', 'photovoltaik-anlagen', 'klimaaktivist',
               'klimaaktivistin', 'klimaaktivisten', 'klimaaktivistinnen', 'klimaaktivismus', 'klimademo',
               'klimademonstration', 'klima-demo', 'klima-demonstration', 'klimademonstrant',
               'klimademonstranten', 'klima-demonstrant', 'klima-demonstranten'],
    
    'politikEuropa' : ['eu', 'europa', 'europäische', 'europäischen', 'europäischer', 'europäisches', 'european',
                       'europeanunion', 'europe', 'kommission', 'europarat', 'brexit', 'europawahl',
                       'ewr', 'eurozone', 'ezb', 'eugh', 'schengen', 'europapolitik', 'ministerrat'],
    
    'politikInternational' : ['außenpolitik', 'außenministerin', 'baerbock', 'diplomatie', 'usa', 'china', 'un',
                              'uno', 'nato', 'vereinte', 'nationen', 'unitednations', 'international',
                              'internationale'],
    
    'politikNational' : ['ampel', 'ampelkoalition', 'verticaltrafficlight', 'koalitionsvertrag', 'koalition',
                         'bundesregierung', 'regierung', 'minister', 'mehrheit', 'bundestag', 'ausschuss',
                         'gesetzentwurf', 'abstimmung', 'ausschüsse', 'bundespräsident', 'bundeskanzler',
                         'kanzler', 'länder', 'ländern', 'ostdeutschland', 'opposition', 'debatte', 'wahl',
                         'wahlkampf', 'gewählt', 'wählen', 'wähler', 'wählerin', 'wählerinnen', 'partei',
                         'fraktion', 'spd', 'grüne', 'grünen', 'fdp', 'union', 'cdu', 'csu', 'afd', 'linke',
                         'linken', 'dielinke', 'ssw'],
    
    'verteidigungspolitik' : ['zeitenwende', 'modernisieren', 'modernisierung', 'verteidigungspolitik',
                              'verteidigungsministerin', 'verteidigungsminister', 'verteidigungsministerium',
                              'sondervermögen', 'bundeswehr', 'truppe', 'heer', 'lambrecht', 'pistorius',
                              'verteidigungsfähig', 'verteidigungsfähigkeit', 'landesverteidigung'],
    
    'flutAhrtal' : ['flut', 'flutkatastrophe', 'ahrtal', 'ahr', 'neuenahr', 'ahrweiler', 'lewentz', 'hochwasser'],
    
    'polizistenmordKusel' : ['polizistenmord', 'polizistenmorde', 'polizistenmorden', 'kusel',
                             'mayweilerhof', 'ulmet'],
    
    'sozialesWohnen' : ['wohnungsbau', 'wohnen', 'bauen', 'mieter', 'bauen', 'mieterinnen', 'mieterin', 'miete',
                        'mietendeckel', 'mietenstopp', 'sozialwohnung', 'sozialwohnungen'],
    
    'sozialesEntlastungen' : ['inflation', 'sorgen', 'preise', 'energiepreise', 'lebenshaltungskosten',
                              'lebenshaltung', 'entlasten', 'entlastung', 'entlastungen', 'entlastungspolitik',
                              'einkommen', 'heizkostenzuschuss', '9-euro-ticket', '9euroticket', '9€ticket',
                              'progression', 'armut', 'doppelwumms', 'doppel-wumms'],
    
    'sozialesKinder' : ['kinder', 'kindern', 'schule', 'schulen', 'eltern', 'kinderarmut',
                        'kindergrundsicherung', 'bildung'],
    
    'sozialesAllgemein' : ['mindestlohn', 'übergewinnsteuer', 'schuldenbremse', 'armut', 'sorgen',
                           'aufstocken', 'aufstocker', 'aufstockerin', 'hartziv', 'hartz4', 'bürgergeld',
                           'energiegeld', 'ungleichheit', 'ungerechtigkeit', 'arbeitslosigkeit',
                           'erwerbstätige', 'erwerbstätig', 'arbeitslos', 'arbeitslose', 'arbeitslosenquote'],
    
    'energieNordstream' : ['pipeline', 'nordstream', 'nordstream2', 'klimafonds', 'klimastiftung',
                           'klimastiftungmv', 'gazprom-stiftung', 'nordstream1', 'nordstreamii',
                           'nordstreami', 'ostseepipeline'],
    
    'energieAtomdiskurs' : ['atomkraft', 'atomkraftwerk', 'atomkraftwerke', 'kernkraft', 'kernenergie',
                            'kernkraftwerk', 'kernkraftwerke', 'akw', 'akws', 'netz', 'nuclear',
                            'laufzeitverlängerung', 'laufzeitverlaengerung', 'isar2', 'atomenergie'],
    
    'energieAllgemein' : ['energiepolitik', 'energie', 'gas', 'strom', 'öl',
                          'erneuerbare', 'erneuerbaren', 'energien', 'ctkwh', 'inflation', 'raffinerie',
                          'flüssiggas', 'lng', 'terminal', 'erdgas', 'erdöl',
                          'kohle', 'kohlekraftwerke', 'fracking', 'fossile', 'fossilen', 'pck', 'schwedt'],
    
    'energieKrise' : ['energiekrise', 'gaskrise', 'gasspeicher', 'gaspreis', 'gaspreise', 'versorgungssicherheit',
                      'energieknappheit', 'ölkrise', 'ölpreis', 'ölpreise', 'wärmepreisbremse',
                      'strompreisbremse', 'gaspreisbremse', 'gasumlage', 'energiekosten', 'gaspreisdeckel',
                      'energielieferungen', 'energiekosten', 'energiesicherheit', 'doppelwumms', 'doppel-wumms'],
    
    'ukraineBewaffnung' : ['waffen', 'waffenlieferungen', 'panzer', 'kampfpanzer', 'schützenpanzer', 'gepard',
                           'marder', 'fuchs', 'leopard', 'munition', 'militärhilfe', 'panzerhaubitzen'],
    
    'ukraineReaktion' : ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine',
                         'mariupol', 'butscha', 'saporischschja', 'cherson', 'besatzer', 'aggressor',
                         'völkerrechtswidrig', 'völkerrechtswidriger', 'völkerrechtswidrigen',
                         'völkerrechtswidrigem', 'überfall'],
    
    'ukraineAllgemein' : ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia',
                          'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj',
                          'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato'],
    
    'covidVirus' : ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'longcovid', 'omikron',
                    'delta', 'gesundheitsminister', 'lauterbach', 'gesundheitssystem', 'epidemie', 'corona-krise'],
    
    'covidImpfung' : ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer',
                      'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe',
                      'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen',
                      'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin',
                      'impfgegnerinnen', 'impfzwang'],
    
    'covidInfektionsgeschehen' : ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage',
                                  'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten', 'triage',
                                  'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung',
                                  'tröpfcheninfektion'],
    
    'covidMaßnahmen' : ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren', 'sperrstunde',
                        'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich',
                        'distancing', 'lockdown', 'shutdown', 'ffp2', 'ffp2-maske', 'ffp2-masken', 'quarantäne',
                        'isolation', 'selbstisolation', 'isolieren', 'schutzmaßnahmen', 'coronatest',
                        'coronatests', 'corona-test', 'corona-tests'],
    
    'verkehrAutomobil' : ['pkw', 'auto', 'autos', 'automobil', 'adblue', 'efuel', 'efuels', 'e-fuel', 'e-fuels',
                          'elektromobilität', 'emobilität', 'e-mobilität', 'emobility', 'e-mobility', 'e-auto',
                          'e-fahrzeug', 'e-autos', 'e-fahrzeuge', 'eauto', 'efahrzeug', 'eautos', 'efahrzeuge',
                          'elektroauto', 'elektroautos', 'plugin', 'ladeinfrastruktur', 'ladesäule', 'ladesäulen',
                          'autofahrende', 'autofahrer', 'autofahrerin', 'autofahrerinnen', 'antriebswende',
                          'automobilhersteller', 'verbrenner', 'verbrennungsmotor', 'verbrennerverbot',
                          'verkehrsverbund', 'verkehrsverbünde', 'iaa', 'autobahn', 'autobahnen', 'tempolimit',
                          'sonntagsfahrverbot', 'sprit', 'benzin', 'autofrei', 'autofreie'],
    
    'verkehrÖpnv' : ['öpnv', 'bus', 'busse', 'bahn', 'db', 'bahnhof', 'bahnhöfe', 'zug', 'züge', 'schiene',
                     'schienennetz', 'neuneuroticket', 'neuneurotickets', '9euroticket', '9-euro-ticket',
                     '9€ticket', '9eurotickets', '9-euro-tickets', '9€tickets', 'deutschlandticket',
                     'deutschlandtickets', 'anbindung', 'öpnv-nutzer', 'öpnv-nutzerin', 'öpnv-nutzende',
                     'bahnfahrende', 'zugfahrende', 'nahverkehr', 'deutschlandtakt', 'deutschland-takt',
                     'verspätung', 'verspätungen', 'zugausfall', 'zugausfälle', 'schienenersatzverkehr',
                     'deutschebahn'],
    
    'verkehrPolitik' : ['verkehr', 'verkehrswende', 'verkehrsminister', 'verkehrsministerium',
                        'verkehrsministers', 'verkehrsministeriums', 'mobilitätswende',
                        'bundesverkehrsminister', 'bundesverkehrsministerium', 'bundesverkehrsministers',
                        'bundesverkehrsministeriums'],
    
    'verkehrFahrrad' : ['radfahrer', 'radfahrerin', 'radfahrerinnen', 'radfahren', 'fahrradfahrer',
                        'fahrradfahrerin', 'fahrradfahrerinnen', 'fahrrad', 'fahrradweg', 'fahrradwege',
                        'radweg', 'radwege', 'radfahrende', 'fahrradfreundlich', 'fahrradfreundliche',
                        'radfreundlich', 'radfreundliche', 'fahrradparkhäuser', 'fahrradstellplätze',
                        'radfahrens', 'fahrradfahrens', 'lastenrad', 'lastenfahrrad', 'lastenräder',
                        'lastenfahrräder']
}

In [4]:
dfsThemenliste = []
# stemmer=Stemmer.Stemmer('de')

for key,values in themenliste.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsThemenliste.append(words)

termsThemenliste = pd.concat(dfsThemenliste)
termsThemenliste

,category,term
0,protesteIran,iran
1,protesteIran,iraner
2,protesteIran,iranerin
3,protesteIran,iranerinnen
4,protesteIran,iranisch
...,...,...
20,covidMaßnahmen,schutzmaßnahmen
21,covidMaßnahmen,coronatest
22,covidMaßnahmen,coronatests
23,covidMaßnahmen,corona-test


In [5]:
# Take the 'term' column from our list of terms
termlistThemenliste = list(set(termsThemenliste.term))

# binary=True oder False: bei True werden nur 1 oder 0 gemeldet wenn Wörter vorhanden sind
# vocabulary: die liste mit wörtern die erkannt werden sollen
vectorizer = TfidfVectorizer(binary=True, vocabulary = termlistThemenliste)

## BADEN-WÜRTTEMBERG

In [6]:
polBawü = polBawü.drop(columns="...1")

In [7]:
# categPolBawü = {
#     'protesteIran' : ['iran', 'iraner', 'iranerin', 'iranerinnen', 'iranisch', 'iranische', 'iranischen', 'mullah', 'mullahs', 'mullah-regime', 'scharia', 'jin', 'jiyan', 'azadi', 'jina', 'mahsa', 'amini', 'mahsaamini', 'kopftuch', 'hijab', 'hidschab', 'jinjiyanazadi', 'frauenlebenfreiheit', 'womenlifefreedom', 'internetblockade', 'freeiran', 'iranprotests', 'iranprotests2022', 'iranrevolution2022'],
    
#     'covid' : ['impfpflicht', 'lauterbach', 'corona', 'pandemie'],
        
#     'energiekrise' : ['habeck', 'energie', 'gas', 'klimaschutz', 'strom', 'energiewende', 'erneuerbaren',
#                       'atomkraft', 'energien', 'klima', 'netz', 'kernkraftwerke', 'energieversorgung',
#                       'klimakrise', 'akw'],
        
#     'ukraine' : ['ukraine', 'russland', 'putin', 'krieg', 'sanktionen', 'putins', 'russia', 'russlands',
#                  'angriffskrieg', 'waffenlieferungen', 'russischen', 'standwithukraine', 'russische',
#                  'kriegsverbrechen', 'angriff'],
        
#     'politikAllgemein' : ['afd', 'spd', 'politik', 'grüne', 'cdu', 'grünen', 'partei', 'bürger',
#                           'strobl', 'opposition', 'kretschmann', 'mdb', 'parteien', 'merkel',
#                           'unserlandzuerst', 'innenminister'],
        
#     'politikWahl' : ['erfolg', 'wahl', 'ergebnis', 'zusammenarbeit', 'aufgabe'],
        
#     'sozialpolitik' : ['bürgergeld', 'entlastungspaket', 'entlastung',
#                        'mindestlohn', 'entlastungen', 'entlasten'],
        
#     'politikInternational' : ['europa', 'welt', 'europeanunion', 'nato', 'france', 'usa', 'europäische',
#                               'frankreich'],
        
#     'freiheitsrechte' : ['menschenrechte', 'pressefreiheit'],
        
#     'öpnvVerkehr' : ['bahn', 'mobilität', 'öpnv', 'infrastruktur', 'verkehr', '9euroticket', 'schiene',
#                      'bus', 'auto', 'straße'],
# }

In [8]:
matrixBawü = vectorizer.fit_transform(polBawü.text.astype('U'), )
words_df_Bawü = pd.DataFrame(matrixBawü.toarray(), columns = vectorizer.get_feature_names_out())

In [9]:
# Group the terms by category, then loop through each category
# Begriffe nach kategorie gruppieren und durch jede kategorie durchiterieren
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polBawü[category_name] = words_df_Bawü[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion'] 

Topic: covidMaßnahmen 
Worte: ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren', 'sperrstunde', 'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich', 'distancing', 'lockdown', 'shutdown', 'ffp2', 'ffp2-maske', 'ffp2-masken', 'quarantäne', 'isolation', 'selbstisolation', 'i

In [10]:
polBawü.groupby('user').covidVirus.sum().sort_values(ascending=False)

user
JuergenBraunAfD    75
MalteKaufmann      66
DrChristinaBaum    57
Alice_Weidel       29
HoffmannForest     24
                   ..
MichaelHennric2     0
SteffenBilger       0
NZippelius          0
PascalKober         0
AWidmannMauz        0
Name: covidVirus, Length: 79, dtype: int64

In [11]:
polBawü.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,polizistenmordKusel,protesteIran,sozialesAllgemein,sozialesEntlastungen,sozialesKinder,sozialesWohnen,ukraineAllgemein,ukraineBewaffnung,ukraineReaktion,verteidigungspolitik
user,,,,,,,,,,,,,,,,,,,,,
AWidmannMauz,2022-10-26T12:48:19Z2022-10-23T17:43:48Z2022-1...,1.401058e+20,Die angekündigten Sanktionen des #Iran gegen @...,215,230,1267,29,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/vmedick/status/15820436974...,...,0,4,1,1,1,0,14,1,4,0
AkbulutGokay,2022-10-31T08:18:14Z2022-10-30T11:59:55Z2022-1...,2.536233e+20,Herzlichen Glückwunsch #Lula! Dies ist ein Sie...,1658,2032,7010,172,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BaharAslan_/status/1586646...,...,0,5,6,11,6,1,13,3,1,1
Alice_Weidel,2022-10-31T15:37:59Z2022-10-31T15:35:59Z2022-1...,3.472247e+20,Und auch heute verweigert sich die #EZB echten...,99135,205809,1031372,9229,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BILD/status/15443120676787...,...,0,1,2,17,0,0,40,8,7,2
AnjaReinalter,2022-05-13T18:59:13Z,1.525189e+18,wenn wir es ernst meinen mit der Gleichwertigk...,1,1,10,0,0,0.0,https://twitter.com/GrueneBundestag/status/152...,...,0,0,0,0,1,0,0,0,0,0
ChrisKuehn_mdb,2022-10-28T11:08:55Z2022-10-27T16:23:13Z2022-1...,2.151907e+20,"Großer Erfolg für Klima, Menschen &amp; Umwelt...",213,567,2608,29,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/KlimaschutzU/status/158266...,...,0,0,5,9,3,4,10,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rischwasu,2022-10-28T16:44:53Z2022-10-23T18:51:26Z2022-1...,1.597405e+20,Null Toleranz für solche Taten und auch nicht ...,282,215,1056,45,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/SZ_Politik/status/15860326...,...,1,0,1,1,1,0,6,0,0,0
s_aeffner,2022-10-31T13:23:52Z2022-10-31T13:23:51Z2022-1...,2.538288e+20,4/4 Gesellschaft getragen. Das war falsch! Den...,1411,597,2205,65,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/phoenix_de/status/15804961...,...,0,0,15,22,9,1,14,0,8,3
sebastian_es_,2022-10-19T13:58:52Z2022-10-19T12:14:18Z2022-1...,1.311661e+20,Den Appell gibt‘s hier: https://t.co/5tUXGtrZa...,241,394,3139,29,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/c_lindner/status/157289724...,...,0,0,5,21,1,0,9,0,2,8


In [12]:
polBawüCategorized = polBawü
polBawüCategorized.to_csv("../../methods/representative-speech/politikerdaten/BAWÜ_politikerKategorisiert.csv")

## BAYERN

In [13]:
polBay = polBay.drop(columns='...1')

In [14]:
# categPolBay = {
#     'covid' : ['impfpflicht', 'corona', 'pandemie', 'lauterbach', 'maskenpflicht', 'impfung', 'impfen',
#                'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'masken', 'maske', 'rki', 'robert-koch-institut',
#                'inzidenz', 'neuinfektionen', 'corona-fälle', 'lockerungen', 'corona-regeln', 'corona-lockerungen',
#                'impfstoff', 'geimpft'],
    
#     'politikEuropa' : ['welt', 'europeanunion', 'europäische',
#                        'eu', 'europa', 'kommission', 'eu-kommission', 'europäisch', 'europäisches', 'europäischen'],
    
#     'politikBund' : ['bundesregierung', 'scholz', 'berlin', 'germany', 'regierung', 'kanzler', 'bundeskanzler', 'bundestag', 'ampel', 'union', 'ampelkoalition', 'merz'],
    
#     'ukraine' : ['ukraine', 'russland', 'putin', 'krieg', 'waffen', 'russischen', 'sanktionen', 'angriffskrieg', 'putins', 'waffenlieferungen', 'russia', 'nato', 'standwithukraine', 'russische'],
    
#     'investitionFinanzen' : ['mrd', 'schuldenbremse', 'geld', '€', 'milliarden', 'mio', 'sondervermögen', 'millionen'],
    
#     'cannabis' : ['cannabis', 'legalisierung', 'cannabis-legalisierung'],
    
#     'energiekrise' : ['habeck', 'gas', 'energie', 'atomkraft', 'strom', 'ausbau', 'erneuerbaren', 'energien', 'kernkraftwerke', 'akws', 'öl', 'netz', 'energiekrise'],
    
#     'sozialpolitik' : ['inflation', 'entlastung', 'gasumlage', 'kosten', 'preise', 'entlasten', 'entlastungspaket', 'einkommen', 'energiepreise', 'steuern', 'entlastungen'],
    
#     'politikAllgemein' : ['afd', 'politik', 'spd', 'fdp', 'csu', 'söder', 'partei', 'grüne', 'grünen', 'cdu', 'regierung', 'linke', 'linken']
# }

In [15]:
matrixBayern = vectorizer.fit_transform(polBay.text.astype('U'), )
words_df_Bayern = pd.DataFrame(matrixBayern.toarray(), columns = vectorizer.get_feature_names_out())

In [16]:
# Group the terms by category, then loop through each category
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polBay[category_name] = words_df_Bayern[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion'] 

Topic: covidMaßnahmen 
Worte: ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren', 'sperrstunde', 'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich', 'distancing', 'lockdown', 'shutdown', 'ffp2', 'ffp2-maske', 'ffp2-masken', 'quarantäne', 'isolation', 'selbstisolation', 'i

In [17]:
polBay.groupby('user').covidImpfung.sum().sort_values(ascending=False)

user
UllmannMdB       70
DJanecek         30
AfDProtschka     27
yooHannes        24
CarmenWegge      21
                 ..
TErndl            0
JanPlobner        0
Hoffmann_MdB      0
WagenerNiklas     0
Mehli             0
Name: covidImpfung, Length: 85, dtype: int64

In [18]:
polBay.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,polizistenmordKusel,protesteIran,sozialesAllgemein,sozialesEntlastungen,sozialesKinder,sozialesWohnen,ukraineAllgemein,ukraineBewaffnung,ukraineReaktion,verteidigungspolitik
user,,,,,,,,,,,,,,,,,,,,,
AfDProtschka,2022-10-31T22:34:17Z2022-10-31T17:28:26Z2022-1...,1.183787e+21,"Wenn es nach mir ginge, würde ich die Aktivist...",17894,28028,149501,1275,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/pleiteticker/status/158456...,...,1,1,2,7,17,2,34,9,2,4
AlexanderRadwan,2022-10-27T10:36:11Z2022-10-27T10:18:31Z2022-1...,4.539013e+19,Was alleine in den letzten Tagen an fatalen au...,76,81,497,11,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AlexanderRadwan/status/158...,...,0,0,0,0,0,0,10,0,0,5
AloisRainer,2022-09-19T15:27:19Z2022-09-08T18:06:34Z2022-0...,1.086731e+19,Heute waren Vertreter der Finanzmarktaufsicht ...,3,12,18,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,0,0,0,1,0,0,0,0,0,0
AndreaLindholz,2022-10-26T12:20:01Z2022-10-25T16:05:35Z2022-1...,1.818860e+20,#HamburgerHafen: Falsche Entscheidung! Damit d...,578,444,2616,44,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/cducsupm/status/1579825218...,...,0,2,0,0,10,0,20,2,2,8
AtesGuerpinar,2022-10-28T17:04:08Z2022-10-28T16:30:50Z2022-1...,1.371148e+21,#Söder will keine #Drogen in #Bayern. Da bleib...,3672,8551,40924,502,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/JanaFrielingha1/status/158...,...,0,0,30,78,18,4,14,12,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
smuellermdb,2022-10-31T15:35:38Z2022-10-31T15:34:58Z2022-1...,2.214237e+20,@csu_bt jack-o-lantern Eine Regierung zum Gru...,6448,3939,31954,558,[Gif(thumbnailUrl='https://pbs.twimg.com/tweet...,0.0,https://twitter.com/csu_bt/status/158710102275...,...,0,1,1,6,1,2,29,2,2,5
stamm_fibich,2022-10-26T12:31:42Z2022-10-13T15:12:30Z2022-1...,7.077767e+19,"Zum Eckpunktepapier ""Einführung einer kontroll...",163,134,518,30,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/RND_de/status/151901469692...,...,0,0,0,0,1,0,1,0,0,0
th_sattelberger,2022-10-28T15:45:39Z2022-10-28T10:31:48Z2022-1...,7.942284e+20,@FresowR @frau_suse @LyllithB Dann Google mal ...,5605,3472,29158,814,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/stifterverband/status/1584...,...,0,4,1,2,18,0,20,0,6,1


In [19]:
polBayCategorized = polBay
polBayCategorized.to_csv("../../methods/representative-speech/politikerdaten/BAYERN_politikerKategorisiert.csv")

## BERLIN

In [20]:
polBerlin = polBerlin.drop(columns='...1')

In [21]:
# categPolBerlin = {
#     'ukraine' : ['ukraine', 'krieg', 'putin', 'russland', 'sanktionen', 'angriffskrieg', 'standwithukraine', 'waffen', 'russischen', 'putins', 'russische'],
    
#     'proteste' : ['leben', 'demokratie', 'frauen', 'freiheit', 'solidarität', 'gedanken', 'hass', 'rassismus', 'iran', 'menschenrechte', 'gedenken'],
    
#     'politikAllgemein' : ['afd', 'spd', 'grüne', 'fdp', 'linke', 'grüne', 'partei', 'parteien', 'rede', 'bundesregierung', 'debatte', 'scholz', 'merz', 'antrag', 'land', 'noafd'],
    
#     'sozialpolitikEntlastung' : ['entlastung', 'einkommen', 'übergewinnsteuer', 'entlastet', 'entlastungspaket', 'heizkostenzuschuss'],
    
#     'bürgeradressierung' : ['bürger', 'wahlkreis', 'diskussion', 'bürgerinnen', 'anliegen', 'austausch', 'gespräch'],
    
#     'covid' : ['corona', 'covid', 'covid19', 'coronavirus', 'virus', 'impfung', 'impfen', 'geimpft', 'impfpflicht', 'impfstoff', 'pandemie', 'maske', 'masken', 'maskenpflicht', 'lauterbach', 'gesundheitsminister', 'rki', 'robert-koch-institut', 'neuinfektionen', 'corona-fälle', 'lockerungen', 'corona-regeln', 'corona-lockerungen'],
    
#     'sozialpolitikBildungKinder' : ['kinder', 'schulen', 'kindern', 'bildung', 'eltern', 'kindergrundsicherung', 'armut', 'schule'],
    
#     'sozialpolitikWohnen' : ['wohnen', 'bauen', 'neubau', 'vorkaufsrecht', 'mieterinnen', 'mieter', 'mieterin', 'vermieter', 'vermieterin', 'miete', 'wohnungsbau']
# }

In [22]:
matrixBerlin = vectorizer.fit_transform(polBerlin.text.astype('U'), )
words_df_Berlin = pd.DataFrame(matrixBerlin.toarray(), columns = vectorizer.get_feature_names_out())

In [23]:
# Group the terms by category, then loop through each category
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polBerlin[category_name] = words_df_Berlin[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion'] 

Topic: covidMaßnahmen 
Worte: ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren', 'sperrstunde', 'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich', 'distancing', 'lockdown', 'shutdown', 'ffp2', 'ffp2-maske', 'ffp2-masken', 'quarantäne', 'isolation', 'selbstisolation', 'i

In [26]:
polBerlin.groupby('user').covidImpfung.sum().sort_values(ascending=False)

user
ninastahr          48
GtzFrmming         28
Beatrix_vStorch    24
JM_Luczak          16
AnAudretsch        14
LFLindemann        10
HanSteinmueller     7
lisapaus            6
stuewer             6
Ottilie_Klein       5
helmut_kleebank     4
RenateKuenast       4
GregorGysi          4
LoetzschMdB         2
ThomasHeilmann      2
Meyer_FDP           2
PetraPauMaHe        1
MarioCzaja          1
LieblingXhain       1
GottfriedCurio      1
AnniKlose           0
StefanGelbhaar      0
DanielaKluckert     0
CanselK             0
pascalmeiser        0
HakanDemirNK        0
Name: covidImpfung, dtype: int64

In [28]:
polBerlin.groupby('user').sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,polizistenmordKusel,protesteIran,sozialesAllgemein,sozialesEntlastungen,sozialesKinder,sozialesWohnen,ukraineAllgemein,ukraineBewaffnung,ukraineReaktion,verteidigungspolitik
user,,,,,,,,,,,,,,,,,,,,,
AnAudretsch,2022-10-30T12:44:40Z2022-10-27T20:15:24Z2022-1...,7.053402e+20,@_FriedrichMerz Kompetente CDU-Staatssekretäre...,22638,10895,84293,2113,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AnAudretsch/status/1580487...,...,0,1,36,52,9,4,52,0,10,3
AnniKlose,2022-10-18T16:41:52Z2022-07-23T13:40:16Z2022-0...,5.205168e+19,Fragen oder Anmerkungen zum neuen #Buergergeld...,299,136,1165,24,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/Forum_DL21/status/15822675...,...,0,0,7,7,0,1,3,0,2,0
Beatrix_vStorch,2022-10-31T14:53:09Z2022-10-30T22:24:22Z2022-1...,5.867109e+20,Ohne die stete Verleihung des deutschen Passes...,84159,90954,513428,5456,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/PapkeGerhard/status/158702...,...,0,7,6,14,7,2,11,2,4,7
CanselK,2022-10-31T13:27:17Z2022-10-31T13:27:17Z2022-1...,5.382520e+20,@BNBundestag @KarambaDiaby @erikvonmalottki @B...,1634,2371,12755,254,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/LionHirth/status/158340204...,...,0,0,17,28,5,13,8,0,7,2
DanielaKluckert,2022-10-28T08:38:21Z2022-10-28T08:29:55Z2022-1...,1.621221e+20,Das ist verantwortungsbewusste Finanzpolitik. ...,644,367,2688,69,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/c_lindner/status/158591281...,...,0,0,0,13,1,0,11,0,3,2
GottfriedCurio,2022-09-28T14:12:39Z2022-07-19T09:30:02Z2022-0...,1.511723e+19,Heute hat die Bundesregierung den Innenausschu...,515,1416,5466,85,0,0.0,https://twitter.com/NancyFaeser/status/1483846...,...,0,0,0,0,0,0,0,1,0,0
GregorGysi,2022-10-31T11:34:43Z2022-10-31T11:34:42Z2022-1...,4.235336e+20,Ich hoffe inständig auf einen progressiven Auf...,26981,22220,177935,4228,[Video(thumbnailUrl='https://pbs.twimg.com/med...,0.0,https://twitter.com/LulaOficial/status/1586852...,...,0,2,12,12,4,0,74,22,14,14
GtzFrmming,2022-10-31T17:04:48Z2022-10-31T00:49:08Z2022-1...,1.514943e+21,"Vergesst nicht, wem ihr den #Feiertag zu verda...",9435,17792,69476,1410,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/berlinerzeitung/status/158...,...,0,5,2,12,64,2,52,7,13,7
HakanDemirNK,2022-10-22T17:43:47Z2022-10-21T08:15:40Z2022-1...,3.101996e+20,"Nach der Unteilbar-Demo 2018 die größte Demo, ...",1259,1343,13252,135,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/NancyFaeser/status/1578025...,...,0,5,16,13,8,2,24,0,7,1


In [27]:
polBerlinCategorized = polBerlin
polBerlinCategorized.to_csv("../../methods/representative-speech/politikerdaten/BERLIN_politikerKategorisiert.csv")

## BRANDENBURG

In [29]:
polBrandenburg = polBrandenburg.drop(columns='...1')

In [30]:
# categPolBrandenburg = {
#     'politikAllgemein' : ['afd', 'bundestag', 'bundesregierung', 'impfpflicht', 'bürger', 'austausch', 'alternativefürdeutschland', 'fraktion'],
    
#     'covid' : ['corona', 'covid', 'covid19', 'coronavirus', 'virus', 'impfung', 'impfen', 'impfpflicht', 'impfstoff', 'geimpft', 'pandemie', 'maske', 'masken', 'maskenpflicht', 'lauterbach', 'gesundheitsminister', 'rki', 'neuinfektionen', 'corona-fälle', 'lockerungen', 'corona-lockerungen', 'corona-regeln'],
    
#     'ukraine' : ['ukraine', 'krieg', 'russland', 'putin', 'russ', 'putins'],
    
#     'energiepolitik' : ['energie', 'öl', 'schwedt', 'ausbau', 'pck',
#                         'gas', 'akw', 'atomkraftwerk', 'akws', 'atomkraftwerke', 'raffinerie', 'lng', 'erdgas', 'erdöl']
# }

In [31]:
matrixBrandenburg = vectorizer.fit_transform(polBrandenburg.text.astype('U'), )
words_df_Brandenburg = pd.DataFrame(matrixBrandenburg.toarray(), columns = vectorizer.get_feature_names_out())

In [32]:
# Group the terms by category, then loop through each category
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polBrandenburg[category_name] = words_df_Brandenburg[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion'] 

Topic: covidMaßnahmen 
Worte: ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren', 'sperrstunde', 'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich', 'distancing', 'lockdown', 'shutdown', 'ffp2', 'ffp2-maske', 'ffp2-masken', 'quarantäne', 'isolation', 'selbstisolation', 'i

In [38]:
polBrandenburg.groupby('user').energieNordstream.sum().sort_values(ascending=False)

user
SteffenKotre      8
Rene_Springer     4
MiKellner         3
NKleinwaechter    2
anked             2
LindaTeuteberg    2
goerke_c          1
sonjaeichwede     0
OlafScholz        0
ABaerbock         0
ArianeFaescher    0
M_Papendieck      0
LehmannSpd        0
Krawallstein      0
JanaSchimke       0
Hannes_Gnauck     0
zierke            0
Name: energieNordstream, dtype: int64

In [39]:
polBrandenburg.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,polizistenmordKusel,protesteIran,sozialesAllgemein,sozialesEntlastungen,sozialesKinder,sozialesWohnen,ukraineAllgemein,ukraineBewaffnung,ukraineReaktion,verteidigungspolitik
user,,,,,,,,,,,,,,,,,,,,,
ABaerbock,2022-10-30T13:08:20Z2022-10-30T13:08:20Z2022-1...,3.549096e+20,Ob Menschen ihre nächste Mahlzeit bezahlen kön...,160650,93666,996927,15272,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/10DowningStreet/status/156...,...,0,12,1,2,9,1,84,1,17,0
ArianeFaescher,2022-04-10T21:31:14Z2022-03-15T17:49:46Z,3.017059e+18,Auch Politiker erleben als Menschen persönlich...,5,2,12,0,0,0.0,0,...,0,0,0,0,0,0,1,0,0,0
Hannes_Gnauck,2022-10-22T20:51:09Z2022-10-22T14:51:46Z2022-1...,1.173205e+20,@MatthiasHoehne1 @NancyFaeser *seidIn #Ludwigs...,1255,2778,12494,102,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/larafritzsche/status/15805...,...,0,0,0,1,0,0,2,1,3,4
JanaSchimke,2022-10-30T17:43:05Z2022-10-24T21:53:35Z2022-1...,1.964963e+20,Morgen wieder @BILD einschalten television ...,11560,8775,67443,1437,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BILD/status/15867715401584...,...,0,0,2,8,9,2,9,1,7,3
Krawallstein,2022-10-22T23:26:53Z2022-10-22T22:39:51Z2022-0...,1.437443e+20,Vorfälle dieser Art sind kein Einzelfall. Mut ...,1645,2526,4022,307,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BertaHelm/status/158353030...,...,0,0,1,1,2,0,6,0,0,0
LehmannSpd,2022-01-27T11:12:52Z2022-01-27T11:12:52Z2022-0...,3.863037e+19,Wir als Gesellschaft müssen uns gegen Antisemi...,24,8,48,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,0,0,0,0,2,0,0,0,0,0
LindaTeuteberg,2022-10-30T10:34:15Z2022-10-26T07:32:17Z2022-1...,3.508450e+20,"""Mit Politik kann man keine Kultur machen, abe...",4606,7779,56767,653,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/andikynast/status/15841044...,...,0,0,2,6,1,0,16,3,14,5
M_Papendieck,2022-10-28T16:42:11Z2022-10-28T16:40:19Z2022-1...,2.537158e+20,@FUSSUNDHAND @bundestag_tweet Wirkt nicht so a...,169,242,778,16,[Gif(thumbnailUrl='https://pbs.twimg.com/tweet...,0.0,https://twitter.com/derspiegel/status/15858054...,...,0,0,9,5,4,3,3,1,2,0
MiKellner,2022-10-28T12:02:25Z2022-10-28T08:30:44Z2022-1...,6.372335e+20,Gratuliere lieber Winfried. Ein wichtiger Prei...,5070,3167,33172,481,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/emilybuening/status/158057...,...,0,0,7,19,2,9,33,2,13,0


In [40]:
polBrandenburgCategorized = polBrandenburg
polBrandenburgCategorized.to_csv("../../methods/representative-speech/politikerdaten/BRANDENBURG_politikerKategorisiert.csv")

## BREMEN

Bremen: Problem, da Tweets massiv ungleich auf die wenigen Politiker verteilt, deswegen nur Tweets zur Cannabis-Legalisierung vorhanden.

In [41]:
polBremen = polBremen.drop(columns='...1')

In [42]:
matrixBremen = vectorizer.fit_transform(polBrandenburg.text.astype('U'), )
words_df_Bremen = pd.DataFrame(matrixBremen.toarray(), columns = vectorizer.get_feature_names_out())

In [43]:
# Group the terms by category, then loop through each category
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polBremen[category_name] = words_df_Bremen[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion'] 

Topic: covidMaßnahmen 
Worte: ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren', 'sperrstunde', 'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich', 'distancing', 'lockdown', 'shutdown', 'ffp2', 'ffp2-maske', 'ffp2-masken', 'quarantäne', 'isolation', 'selbstisolation', 'i

In [60]:
polBremen.groupby('user').covidMaßnahmen.sum().sort_values(ascending=False)

user
KirstenKappert    1
RyglewskiS        0
TR_Bremen         0
docredder         0
Name: covidMaßnahmen, dtype: int64

In [51]:
polBremen.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,polizistenmordKusel,protesteIran,sozialesAllgemein,sozialesEntlastungen,sozialesKinder,sozialesWohnen,ukraineAllgemein,ukraineBewaffnung,ukraineReaktion,verteidigungspolitik
user,,,,,,,,,,,,,,,,,,,,,
KirstenKappert,2022-10-27T07:54:26Z2022-10-26T09:44:20Z2022-1...,2.531164e+20,Gesundheits- und Jugendschutz gibt es nur in e...,4493,3663,25166,442,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/SaskiaLeaRaquel/status/158...,...,0,10,1,2,5,1,56,1,14,0
RyglewskiS,2022-10-27T18:36:16Z2022-10-18T11:46:39Z2022-1...,5.501784e+19,Danke @bodoramelow für das gute Gespräch über ...,19,60,210,1,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/thueringende/status/158554...,...,0,0,1,1,1,0,10,0,1,0
TR_Bremen,2022-08-30T08:53:18Z2022-08-29T06:49:26Z2022-0...,9.102598e+19,Gestern wurde der Förderpreis des Musikfestes ...,149,343,1166,63,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/joewagjoe/status/156325208...,...,0,0,0,1,1,0,0,0,0,1
docredder,2022-10-30T11:04:01Z2022-10-26T12:18:27Z2022-1...,1.421367e+20,@au_huur @torstenherbst @WECNuclear Die Zahlen...,1450,1928,20567,150,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/RolandTichy/status/1584970...,...,0,2,0,1,5,0,28,0,4,0


In [61]:
polBremenCategorized = polBremen
polBremenCategorized.to_csv("../../methods/representative-speech/politikerdaten/BREMEN_politikerKategorisiert.csv")

## HAMBURG

In [62]:
polHamburg = polHamburg.drop(columns='...1')

In [63]:
# categPolHamburg = {
#     'energiekrise' : ['gas', 'kernkraftwerke', 'kernenergie',
#                       'versorgungssicherheit', 'energiepolitik', 'öl'],
    
#     'ukraine' : ['ukraine', 'krieg', 'putin', 'russischen', 'russland',
#                  'russlands', 'putins', 'russia', 'angriff', 'nato', 'sanktionen',
#                  'waffenlieferungen', 'russische', 'ukrainischen'],
    
#     'politikNational' : ['verbraucherschutz', 'regierung', 'bundeskanzler', 'bund', 'koalitionsvertrag',
#                          'bundestag', 'impfpflicht', 'rede', 'debatte', 'gesetz', 'politik', 'spd', 'land',
#                          'ampelkoalition', 'grüne', 'ampel', 'bund', 'ländern', 'wahl', 'ampel',
#                          'digitalisierung', 'energiewende', 'sunflower', 'impfpflicht', 'gesellschaft',
#                          'präsident'],
    
#     'sozialpolitik' : ['gasumlage', 'übergewinnsteuer', 'entlastung', 'einkommen', 'entlastet', 'entlastungspaket', 'heizkostenzuschuss']
# }

In [64]:
matrixHamburg = vectorizer.fit_transform(polHamburg.text.astype('U'), )
words_df_Hamburg = pd.DataFrame(matrixHamburg.toarray(), columns = vectorizer.get_feature_names_out())

In [65]:
# Group the terms by category, then loop through each category
for category_name, rows in termsThemenliste.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polHamburg[category_name] = words_df_Hamburg[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern', 'comirnaty', 'risikogruppe', 'risikogruppen', 'herdenimmunität', 'immunität', 'nebenwirkung', 'nebenwirkungen', 'impfschäden', 'impfschaden', 'querdenken', 'querdenker', 'impfgegner', 'impfgegnerin', 'impfgegnerinnen', 'impfzwang'] 

Topic: covidInfektionsgeschehen 
Worte: ['rki', 'robert-koch-institut', 'infektionsgeschehen', 'infektionslage', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten', 'triage', 'reproduktionszahl', 'inkubationszeit', 'tröpfchenübertragung', 'tröpfcheninfektion'] 

Topic: covidMaßnahmen 
Worte: ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren', 'sperrstunde', 'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich', 'distancing', 'lockdown', 'shutdown', 'ffp2', 'ffp2-maske', 'ffp2-masken', 'quarantäne', 'isolation', 'selbstisolation', 'i

In [67]:
polHamburg.groupby('user').energieKrise.sum().sort_values(ascending=False)

user
krusehamburg      57
VriesChristoph    16
kathabeck          9
till_steffen       7
MetinHakverdi      4
christophploss     4
ZaklinNastic       2
ria_schroeder      2
FHoppermann        1
Doro_Martin        0
NielsAnnen         0
emiliafester       0
linda_lobster      0
oezoguz            0
Name: energieKrise, dtype: int64

In [100]:
polHamburg.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,followerAmount,bundesland,einzug,partei,geschlecht,text,energiekrise,politikNational,sozialpolitik,ukraine
user,,,,,,,,,,,,,,,,,,,,,
Doro_Martin,2022-10-31T08:00:52Z2022-10-31T07:52:22Z2022-1...,9.291158e+20,@dankegut smiling_face_with_open_handsWünsche...,2455,1613,15093,217,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/LulaOficial/status/1586852...,...,3015983,HamburgHamburgHamburgHamburgHamburgHamburgHamb...,ErststimmeErststimmeErststimmeErststimmeErstst...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,weiblichweiblichweiblichweiblichweiblichweibli...,@dankegut smilingfacewithopenhandswünsche euc...,0,65,7,11
FHoppermann,2022-10-26T19:51:20Z2022-10-21T07:35:44Z2022-1...,1.972105e+20,Ein absoluter #tiefpunkt des Verhältnisses von...,1078,1389,10324,126,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/NatalieAmiri/status/157623...,...,309987,HamburgHamburgHamburgHamburgHamburgHamburgHamb...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,CDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUC...,weiblichweiblichweiblichweiblichweiblichweibli...,ein absoluter tiefpunkt des verhältnisses von ...,1,51,0,16
MetinHakverdi,2022-10-31T19:08:14Z2022-10-30T11:04:29Z2022-1...,1.236802e+21,Hakverdi vs. Amthor für 45 Minuten. Aber seht ...,1164,1542,10386,90,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/phoenix_de/status/15871286...,...,4146467,HamburgHamburgHamburgHamburgHamburgHamburgHamb...,ErststimmeErststimmeErststimmeErststimmeErstst...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,hakverdi vs amthor für 45 minuten aber seht se...,8,78,4,130
NielsAnnen,2022-10-31T21:21:45Z2022-10-30T22:50:02Z2022-1...,3.350960e+20,Great to see two friends meeting!Ein historisc...,2070,2062,13069,183,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AHuitfeldt/status/15871181...,...,3892694,HamburgHamburgHamburgHamburgHamburgHamburgHamb...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,great to see two friends meetingein historisch...,0,24,1,40
VriesChristoph,2022-10-31T19:21:40Z2022-10-31T08:08:34Z2022-1...,7.906547e+20,Viele Jahre haben #SPD und #Gruene beim #IZH b...,16521,17130,119823,1762,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BornplatzHH/status/1582953...,...,2229472,HamburgHamburgHamburgHamburgHamburgHamburgHamb...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,CDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUC...,männlichmännlichmännlichmännlichmännlichmännli...,viele jahre haben spd und gruene beim izh beid...,10,121,7,88
ZaklinNastic,2022-10-30T17:43:25Z2022-10-26T14:33:14Z2022-1...,1.983165e+20,Die #Bundesregierung verweigert wichtige Infor...,2502,3057,10640,366,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/SWagenknecht/status/158336...,...,717210,HamburgHamburgHamburgHamburgHamburgHamburgHamb...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,Die LinkeDie LinkeDie LinkeDie LinkeDie LinkeD...,weiblichweiblichweiblichweiblichweiblichweibli...,die bundesregierung verweigert wichtige inform...,0,21,3,21
christophploss,2022-10-30T19:11:18Z2022-10-28T11:17:05Z2022-1...,6.550236e+20,"Im #Iran werden Frauen und Männer umgebracht, ...",23180,8903,73661,1979,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/micha_bloss/status/1585711...,...,3651219,HamburgHamburgHamburgHamburgHamburgHamburgHamb...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,CDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUC...,männlichmännlichmännlichmännlichmännlichmännli...,im iran werden frauen und männer umgebracht ve...,20,116,5,49
emiliafester,2022-09-27T17:38:29Z2022-09-23T14:35:37Z2022-0...,4.582456e+19,„#Sozialtourismus“ ist NPD-Sprech. Das wa

In [101]:
polHamburgCategorized = polHamburg
polHamburgCategorized.to_csv("../../methods/representative-speech/politikerdaten/HAMBURG_politikerKategorisiert.csv")

## Hessen

In [102]:
polHessen = polHessen.drop(columns='...1')

In [104]:
categPolHessen = {
    'ukraine' : ['ukraine', 'russland', 'putin', 'krieg', 'russia', 'standwithukraine', 'putins',
                 'waffen', 'russischen', 'sanktionen', 'russische', 'angriffskrieg', 'ukrainischen', 'russlands'],
    
    'politikGrundsätze' : ['demokratie', 'freiheit', 'menschenrechte', 'rechtsstaatlichkeit'],
    
    'politikNational' : ['rede', 'bundestag', 'antrag', 'union', 'bildung', 'spd',
                         'verticaltrafficlight', 'cdu', 'scholz', 'spd', 'merz', 'staat'],
    
    'investitionenReformen' : ['klimaschutz', 'euro', 'energiewende', 'bundeswehr', 'ausbau', 'energien',
                               'mrd', 'windkraft', 'herausforderungen', '€', 'kosten', 'schuldenbremse',
                               'erneuerbaren'],
    
    'covid' : ['corona', 'impfpflicht', 'maßnahmen', 'covid19', 'impfungen', 'coronavirus', 'covid19', 'covid', 'virus', 'masken', 'maske', 'maskenpflicht', 'rki', 'robert-koch-institut', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'lockerungen', 'corona-regeln', 'corona-lockerungen', 'impfstoff', 'geimpft'],
    
    'politikEuropa' : ['europeanunion', 'germany', 'france', 'european', 'northmacedonia',
                       'europe', 'cooperation', 'kosovo', 'partner', 'bih', 'international', 'albania'],
    
    'energiekrise' : ['habeck', 'energie', 'inflation', 'atomkraft', 'gas', 'strom', 'akw', 'gasumlage', 'preise']
}

In [105]:
dfsPolHessen = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categPolHessen.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsPolHessen.append(words)

termsPolHessen = pd.concat(dfsPolHessen)
termsPolHessen

,category,term
0,ukraine,ukraine
1,ukraine,russland
2,ukraine,putin
3,ukraine,krieg
4,ukraine,russia
...,...,...
4,energiekrise,gas
5,energiekrise,strom
6,energiekrise,akw
7,energiekrise,gasumlage


In [106]:
# Take the 'term' column from our list of terms
termlistPolHessen = list(set(termsPolHessen.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistPolHessen)

matrix = vectorizer.fit_transform(polHessen.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [107]:
# Group the terms by category, then loop through each category
for category_name, rows in termsPolHessen.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polHessen[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covid 
Worte: ['corona', 'impfpflicht', 'maßnahmen', 'covid19', 'impfungen', 'coronavirus', 'covid19', 'covid', 'virus', 'masken', 'maske', 'maskenpflicht', 'rki', 'robert-koch-institut', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'lockerungen', 'corona-regeln', 'corona-lockerungen', 'impfstoff', 'geimpft'] 

Topic: energiekrise 
Worte: ['habeck', 'energie', 'inflation', 'atomkraft', 'gas', 'strom', 'akw', 'gasumlage', 'preise'] 

Topic: investitionenReformen 
Worte: ['klimaschutz', 'euro', 'energiewende', 'bundeswehr', 'ausbau', 'energien', 'mrd', 'windkraft', 'herausforderungen', '€', 'kosten', 'schuldenbremse', 'erneuerbaren'] 

Topic: politikEuropa 
Worte: ['europeanunion', 'germany', 'france', 'european', 'northmacedonia', 'europe', 'cooperation', 'kosovo', 'partner', 'bih', 'international', 'albania'] 

Topic: politikGrundsätze 
Worte: ['demokratie', 'freiheit', 'menschenrechte', 'rechtsstaatlichkeit'] 

Topic: politikNational 
Worte: ['rede', 'bundestag', 'antrag', 'un

In [111]:
polHessen.groupby('user').covid.sum().sort_values(ascending=False)

user
JoanaCotar         120
El_KaWeh_           54
katjadler           48
PeterHeidtFDP       42
KLeikert            36
W_SK                33
M_HarderKuehnel     22
K_SA                22
starkwatzinger      19
Timon_Gremmels      16
PhiKraemer           9
AnnaLuehrmann        8
meister_schafft      7
JensZSPD             6
Janine_Wissler       5
nouripour            5
borismijatovic       5
bhoffmann_mdb        5
BjoernSimon          5
DeborahDuering       4
alexmuellerfdp       4
ThorstenLieb         2
AwetTesfaiesus       2
ArmandZorn           2
FrankeEdgar          1
Uwe_Schulz_AfD       1
HBraun               1
MiRo_SPD             1
StHeck               0
TillMansmann         0
dieschmidt           0
ArminSchwarz         0
Ingmar_Jung          0
Jan_Nolte_AfD        0
Name: covid, dtype: int64

In [112]:
polHessen.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,partei,geschlecht,text,covid,energiekrise,investitionenReformen,politikEuropa,politikGrundsätze,politikNational,ukraine
user,,,,,,,,,,,,,,,,,,,,,
AnnaLuehrmann,2022-10-26T14:19:08Z2022-10-26T09:22:08Z2022-1...,1.014524e+21,Jedes Zehntel Grad zählt beim Kampf gegen die ...,2183,5076,28134,606,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/UNFCCC/status/158519476257...,...,B90/Die GrünenB90/Die GrünenB90/Die GrünenB90/...,weiblichweiblichweiblichweiblichweiblichweibli...,jedes zehntel grad zählt beim kampf gegen die ...,8,12,70,344,54,16,108
ArmandZorn,2022-10-29T21:06:04Z2022-10-28T19:25:15Z2022-1...,1.733318e+20,@DirkWieseSPD @schneidercar Unverdient.@berlin...,462,836,13710,59,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/demokratieghst/status/1543...,...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,@dirkwiesespd @schneidercar unverdient@berlinb...,2,4,5,0,1,18,7
ArminSchwarz,2022-03-12T19:12:14Z,1.502724e+18,#Kampfhubschrauberregiment #Fritzlar Truppe pr...,3,1,5,1,0,0.0,0,...,CDU,männlich,kampfhubschrauberregiment fritzlar truppe präs...,0,0,0,0,0,0,0
AwetTesfaiesus,2022-10-30T21:02:21Z2022-10-30T19:46:01Z2022-1...,2.588064e+20,Women in politics. Thank you for your inspirin...,1419,1479,15284,131,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AkwugoEmejulu/status/15829...,...,B90/Die GrünenB90/Die GrünenB90/Die GrünenB90/...,weiblichweiblichweiblichweiblichweiblichweibli...,women in politics thank you for your inspiring...,2,0,2,2,0,11,5
BjoernSimon,2022-10-27T08:33:27Z2022-10-26T20:55:58Z2022-1...,3.247765e+20,Nach meinem #Israel-Aufenthalt nehme ich an ei...,352,360,1021,29,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/faznet/status/157479937506...,...,CDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUC...,männlichmännlichmännlichmännlichmännlichmännli...,nach meinem israel-aufenthalt nehme ich an ein...,5,8,14,0,3,58,20
DeborahDuering,2022-10-27T15:58:50Z2022-10-25T12:09:56Z2022-1...,2.526880e+20,Wir dürfen uns nicht selbst von China abhängig...,765,1084,6092,126,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/venro_eV/status/1584549968...,...,B90/Die GrünenB90/Die GrünenB90/Die GrünenB90/...,weiblichweiblichweiblichweiblichweiblichweibli...,wir dürfen uns nicht selbst von china abhängig...,4,3,14,1,6,18,11
El_KaWeh_,2022-10-31T17:55:38Z2022-10-28T19:35:01Z2022-1...,1.336297e+21,Der hessische #Verfassungsschutz ist mit Straf...,3524,3106,20727,264,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/ChristophDegen/status/1582...,...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,der hessische verfassungsschutz ist mit strafa...,54,50,64,6,30,144,34
FrankeEdgar,2022-09-17T09:35:42Z2022-08-11T09:12:59Z2022-0...,1.224041e+19,Hessischen Krankenhaustag in Offenbach.\n\nDer...,49,21,73,2,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,SPDSPDSPDSPDSPDSPDSPDSPD,männlichmännlichmännlichmännlichmännlichmännli...,hessischen krankenhaustag in offenbach\n\nder ...,1,0,2,0,0,2,0
HBraun,2022-10-21T06:15:47Z2022-10-15T17:27:15Z2022-1...,4.631975e+19,hat sich heute im ZDF #MoMa gegen die Finanzie...,551,388,2735,70,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/Karl_Lauterbach/status/157...,...,CDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUC...,männlichmännlichmännlichmännlichmännlichmännli...,hat sich heute im zdf moma gegen die finanzier...,1,4,3,0,1,8,3


In [113]:
polHessenCategorized = polHessen
polHessenCategorized.to_csv("../../methods/representative-speech/politikerdaten/HESSEN_politikerKategorisiert.csv")

## MECKLENBURG-VORPOMMERN

In [3]:
polMeckPomm = polMeckPomm.drop(columns='...1')

In [13]:
categPolMeckpomm = {
    'ukraine' : ['ukraine', 'russland', 'geflüchteten', 'angriffskrieg'],
    
    'verteidigungspolitik' : ['bundeswehr', 'sondervermögen', 'truppen'],
    
    'sozialesAllgemein' : ['inflation', 'entlastungspaket', 'kinder', 'rentner', 'kinderarmut', 'entlastungspaket', 'kinder', 'mindestlohn'],
    
    'covid' : ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'maske', 'masken', 'maskenpflicht', 'rki', 'robert-koch-institut', 'gesundheitsminister', 'lauterbach', 'inzidenz', 'virologe', 'drosten', 'neuinfektionen', 'corona-fälle', 'corona-regeln', 'corona-lockerungen', 'impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft']
}

In [14]:
dfsPolMeckpomm = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categPolMeckpomm.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsPolMeckpomm.append(words)

termsPolMeckpomm = pd.concat(dfsPolMeckpomm)
termsPolMeckpomm

,category,term
0,ukraine,ukraine
1,ukraine,russland
2,ukraine,geflüchteten
3,ukraine,angriffskrieg
0,verteidigungspolitik,bundeswehr
1,verteidigungspolitik,sondervermögen
2,verteidigungspolitik,truppen
0,sozialesAllgemein,inflation
1,sozialesAllgemein,entlastungspaket
2,sozialesAllgemein,kinder


In [15]:
# Take the 'term' column from our list of terms
termlistPolMeckpomm = list(set(termsPolMeckpomm.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistPolMeckpomm)

matrix = vectorizer.fit_transform(polMeckPomm.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [16]:
# Group the terms by category, then loop through each category
for category_name, rows in termsPolMeckpomm.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polMeckPomm[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covid 
Worte: ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'maske', 'masken', 'maskenpflicht', 'rki', 'robert-koch-institut', 'gesundheitsminister', 'lauterbach', 'inzidenz', 'virologe', 'drosten', 'neuinfektionen', 'corona-fälle', 'corona-regeln', 'corona-lockerungen', 'impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft'] 

Topic: sozialesAllgemein 
Worte: ['inflation', 'entlastungspaket', 'kinder', 'rentner', 'kinderarmut', 'entlastungspaket', 'kinder', 'mindestlohn'] 

Topic: ukraine 
Worte: ['ukraine', 'russland', 'geflüchteten', 'angriffskrieg'] 

Topic: verteidigungspolitik 
Worte: ['bundeswehr', 'sondervermögen', 'truppen'] 



In [17]:
polMeckPomm.groupby('user').covid.sum().sort_values(ascending=False)

user
ulschzi           48
DietmarBartsch    12
reinholdmdb        9
ReemAlabali        3
Leif_Erik_Holm     1
annakassautzki     1
jungeinberlin      1
EnricoKomning      0
GruenClaudia       0
arlt_johannes      0
Name: covid, dtype: int64

In [18]:
polMeckPomm.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,followerAmount,bundesland,einzug,partei,geschlecht,text,sozialesAllgemein,ukraine,verteidigungspolitik,covid
user,,,,,,,,,,,,,,,,,,,,,
DietmarBartsch,2022-10-31T11:10:16Z2022-10-30T23:33:22Z2022-1...,6.784419e+20,"Das #Bürgergeld ist alles andere als optimal, ...",19127,10993,83000,2274,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/ZDFheute/status/1586856910...,...,38352725,Mecklenburg-VorpommernMecklenburg-VorpommernMe...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,Die LinkeDie LinkeDie LinkeDie LinkeDie LinkeD...,männlichmännlichmännlichmännlichmännlichmännli...,das bürgergeld ist alles andere als optimal ab...,54,34,20,12
EnricoKomning,2022-10-31T11:29:50Z2022-10-30T20:51:26Z2022-0...,3.190799e+19,Reichtum in den richtigen Händen hat manchmal ...,728,177,940,32,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AfDimBundestag/status/1502...,...,60627,Mecklenburg-VorpommernMecklenburg-VorpommernMe...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,AfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDA...,männlichmännlichmännlichmännlichmännlichmännli...,reichtum in den richtigen händen hat manchmal ...,1,3,1,0
GruenClaudia,2022-10-31T20:37:21Z2022-10-28T08:39:24Z2022-1...,5.173622e+20,@felixsschulz https://t.co/KlLBZyt9ZwSchnapsza...,991,1155,12401,167,[Gif(thumbnailUrl='https://pbs.twimg.com/tweet...,0.0,https://twitter.com/PaulaPiechotta/status/1583...,...,1508211,Mecklenburg-VorpommernMecklenburg-VorpommernMe...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,B90/Die GrünenB90/Die GrünenB90/Die GrünenB90/...,weiblichweiblichweiblichweiblichweiblichweibli...,@felixsschulz httpstcokllbzyt9zwschnapszahl c...,4,13,2,0
Leif_Erik_Holm,2022-10-27T13:27:42Z2022-10-17T19:04:59Z2022-1...,3.685951e+19,"Antwort der Bundesregierung auf meine Anfrage,...",163,259,1227,12,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,237504,Mecklenburg-VorpommernMecklenburg-VorpommernMe...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,AfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDA...,männlichmännlichmännlichmännlichmännlichmännli...,antwort der bundesregierung auf meine anfrage ...,4,1,0,1
ReemAlabali,2022-10-30T09:10:22Z2022-10-21T16:05:24Z2022-1...,3.525186e+20,Eine mutige und starke Frau ist leider von uns...,4538,3480,28547,404,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/IntegrationBund/status/153...,...,2963961,Mecklenburg-VorpommernMecklenburg-VorpommernMe...,ErststimmeErststimmeErststimmeErststimmeErstst...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,weiblichweiblichweiblichweiblichweiblichweibli...,eine mutige und starke frau ist leider von uns...,7,33,3,3
annakassautzki,2022-10-25T09:28:03Z2022-10-23T21:28:19Z2022-1...,1.313326e+20,@k_lenkeit Diese Emotionen… ich kann kaum an m...,189,304,2888,25,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/Bundeskanzler/status/15839...,...,296583,Mecklenburg-VorpommernMecklenburg-VorpommernMe...,ErststimmeErststimmeErststimmeErststimmeErstst...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,weiblichweiblichweiblichweiblichweiblichweibli...,@klenkeit diese emotionen… ich kann kaum an mi...,1,0,0,1
arlt_johannes,2022-10-22T17:25:57Z2022-10-20T15:49:57Z2022-1...,2.095717e+20,@AndreasSteinme4 @jan_bleis @FiS_Heidelberg Al...,477,244,1594,20,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AkaltinFerdi/status/157123...,...,170154,Mecklenburg-VorpommernMecklenburg-VorpommernMe...,ErststimmeErststimmeErststimmeErststimmeErstst...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,@andreassteinme4 @janbleis @fisheidelberg alle...,2,1,17,0
jungeinberlin,2022-10-26T03:53:49Z2022-09-29T13:00:39Z2022-0...,8.953608e+19,Was für ein Scheiß! Auch fünf Tage nach dieser...,729,412,3843,78,[Photo(previewUrl='https://pbs.twimg.com/medi

In [19]:
polMeckPommCategorized = polMeckPomm
polMeckPommCategorized.to_csv("../../methods/representative-speech/politikerdaten/MECKPOMM_politikerKategorisiert.csv")

## NIEDERSACHSEN

In [3]:
polNds = polNds.drop(columns='...1')

In [21]:
categPolNds = {
    'politikNational' : ['ampel', 'spd', 'regierung', 'fdp', 'scholz', 'opposition', 'mehrheit',
                         'kanzler', 'grüne', 'verticaltrafficlight', 'afd', 'bundestag', 'union',
                         'cdu', 'merz', 'debatte', 'impfpflicht', 'parlamentarischen', 'antrag',
                         'verticaltrafficlight', 'ländern', 'bundestag', 'ausschuss', 'koalition'],
    
    'ukraine' : ['ukraine', 'russland', 'standwithukraine', 'putin', 'russischen', 'putins', 'russische',
                 'waffen', 'angriffskrieg', 'sanktionen', 'waffenlieferungen', 'angriff', 'russlands',
                 'ukrainischen'],
    
    'soziales' : ['inflation', 'entlastung', 'entlastungen', 'entlasten', 'einkommen', 'mindestlohn',
                  'gasumlage', 'entlastungspaket'],
    
    'energieAllg' : ['gas', 'energie', 'nuclear', 'atomkraft', 'akw', 'strom', 'energy', 'erneuerbare',
                     'preise', 'klimaschutz', 'wind', 'erneuerbaren', 'solar', 'climateaction',
                     'fossilen', 'netz'],
    
    'politikWahl' : ['grünen', 'wahl', 'ergebnis', 'landtag', 'zusammenarbeit', 'wählen',
                     'landtagswahl', 'linke', 'partei', 'wahlkampf', 'gewählt'],
    
    'gedenken' : ['verantwortung', 'signal', 'opfer', 'vergessen', 'gewalt', 'antisemitismus', 'rassimus'],
    
    'covid' : ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'maske', 'masken', 'maskenpflicht', 'rki', 'robert-koch-institut', 'gesundheitsminister', 'lauterbach', 'inzidenz', 'virologe', 'drosten', 'neuinfektionen', 'corona-fälle', 'corona-regeln', 'corona-lockerungen', 'impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft']

}

In [22]:
dfsPolNds = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categPolNds.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsPolNds.append(words)

termsPolNds = pd.concat(dfsPolNds)
termsPolNds

,category,term
0,politikNational,ampel
1,politikNational,spd
2,politikNational,regierung
3,politikNational,fdp
4,politikNational,scholz
...,...,...
20,covid,impfstoff
21,covid,impfpflicht
22,covid,impfen
23,covid,impfung


In [23]:
# Take the 'term' column from our list of terms
termlistPolNds = list(set(termsPolNds.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistPolNds)

matrix = vectorizer.fit_transform(polNds.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [24]:
# Group the terms by category, then loop through each category
for category_name, rows in termsPolNds.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polNds[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covid 
Worte: ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'maske', 'masken', 'maskenpflicht', 'rki', 'robert-koch-institut', 'gesundheitsminister', 'lauterbach', 'inzidenz', 'virologe', 'drosten', 'neuinfektionen', 'corona-fälle', 'corona-regeln', 'corona-lockerungen', 'impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft'] 

Topic: energieAllg 
Worte: ['gas', 'energie', 'nuclear', 'atomkraft', 'akw', 'strom', 'energy', 'erneuerbare', 'preise', 'klimaschutz', 'wind', 'erneuerbaren', 'solar', 'climateaction', 'fossilen', 'netz'] 

Topic: gedenken 
Worte: ['verantwortung', 'signal', 'opfer', 'vergessen', 'gewalt', 'antisemitismus', 'rassimus'] 

Topic: politikNational 
Worte: ['ampel', 'spd', 'regierung', 'fdp', 'scholz', 'opposition', 'mehrheit', 'kanzler', 'grüne', 'verticaltrafficlight', 'afd', 'bundestag', 'union', 'cdu', 'merz', 'debatte', 'impfpflicht', 'parlamentarischen', 'antrag', 'verticaltrafficlight', 'ländern', 'bundestag', 'ausschuss', 'koa

In [25]:
polNds.groupby('user').gedenken.sum().sort_values(ascending=False)

user
Helge_Limburg      44
sven_kindler       25
JTrittin           24
KonstantinKuhle    22
christianduerr     19
filizgreen         14
J_Pahlke           14
gitta_connemann    12
KatjaKeul          11
AnikoMerten        11
HeidiReichinnek     8
MGrosseBroemer      8
Dr_Pantazis         7
victorperli         7
GeroHocker          5
karo_otte           5
FBsirske            5
axel_knoerig        3
larsklingbeil       3
cj_schroeder        3
BerndWestphal4      2
FalkoMohrs          2
jensbeeck           2
SwantjeMichael1     2
julia_verlinden     1
JoachimWundrak      1
Johann_Saathoff     1
Amira_M_Ali         1
MareikeWulf         1
mseesternpauly      0
stadler_svenja      0
jschraps            0
hubertus_heil       0
AMattfeldt          0
_anja_schulz        0
Thomas_Ehrhorn      0
TKuban96            0
StegemannAlbert     0
StefWenzel          0
PhilippiSpd         0
MdbMuller           0
JakobBla            0
FahimiYasmin        0
FHeiligenstadt      0
FGuentzler          0
Brehe

In [26]:
polNds.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,partei,geschlecht,text,energieAllg,gedenken,politikNational,politikWahl,soziales,ukraine,covid
user,,,,,,,,,,,,,,,,,,,,,
AMattfeldt,2022-10-26T08:11:24Z2022-10-06T05:50:06Z2022-0...,7.799451e+18,2014: Russland bekommt als Dank für die Annek...,6,11,8,0,0,0.0,0,...,CDUCDUCDUCDUCDU,männlichmännlichmännlichmännlichmännlich,2014 russland bekommt als dank für die annekt...,0,0,4,1,1,2,2
Amira_M_Ali,2022-10-23T13:34:48Z2022-10-22T12:55:29Z2022-1...,1.118241e+20,"Mein Reden!Das muss verhindert werden, und zwa...",8100,5829,39440,1313,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/andikynast/status/15841044...,...,Die LinkeDie LinkeDie LinkeDie LinkeDie LinkeD...,weiblichweiblichweiblichweiblichweiblichweibli...,mein redendas muss verhindert werden und zwar ...,9,1,21,6,7,6,3
AnikoMerten,2022-10-30T08:54:01Z2022-10-30T08:52:58Z2022-1...,5.337702e+20,"Joa. Ich auch, aber wenn wir endlich mit der #...",462,471,4249,23,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/ManuelHoeferlin/status/158...,...,FDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPF...,weiblichweiblichweiblichweiblichweiblichweibli...,joa ich auch aber wenn wir endlich mit der zei...,2,11,23,11,2,19,7
AnkeHennigSPD,2022-10-19T10:03:05Z2022-10-13T15:19:51Z2022-0...,3.092430e+19,Ich habe mit dem Vorwärts über mein erstes Jah...,18,30,70,6,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/vorwaerts/status/158265939...,...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,weiblichweiblichweiblichweiblichweiblichweibli...,ich habe mit dem vorwärts über mein erstes jah...,0,0,8,0,2,0,0
BerndWestphal4,2022-10-05T06:10:16Z2022-09-29T13:56:13Z2022-0...,3.035763e+19,Mein Liveinterview heute Morgen um 6.45 Uhr be...,49,45,152,2,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/rbb24Inforadio/status/1577...,...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,mein liveinterview heute morgen um 645 uhr bei...,1,2,8,3,1,1,0
BreherSilvia,2022-09-17T12:32:14Z,1.571115e+18,Ich spreche kein Russisch. Aber ist dies der b...,8,6,18,0,0,0.0,https://twitter.com/adnashmyash/status/1570791...,...,CDU,weiblich,ich spreche kein russisch aber ist dies der bi...,0,0,0,0,0,1,0
Dr_Pantazis,2022-10-31T09:05:29Z2022-10-21T13:31:14Z2022-1...,4.709468e+20,Germany red_heart Brazil Parabéns por sua e...,355,548,1677,49,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/Bundeskanzler/status/15869...,...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,germany redheart brazil parabéns por sua el...,5,7,57,18,7,72,34
FBsirske,2022-10-31T14:51:37Z2022-10-27T15:55:30Z2022-1...,2.048015e+20,"Söder nimmt sich Trump zum Vorbild, indem er F...",837,1275,10403,177,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/EPSocialAffairs/status/158...,...,B90/Die GrünenB90/Die GrünenB90/Die GrünenB90/...,männlichmännlichmännlichmännlichmännlichmännli...,söder nimmt sich trump zum vorbild indem er fa...,10,5,21,8,25,10,9
FGuentzler,2022-04-09T12:58:28Z2022-03-19T14:25:50Z2022-0...,8.998252e+18,@FabioDeMasi So ist esTrotz Dopingverstrickung...,8,4,6,0,0,0.0,0,...,CDUCDUCDUCDUCDUCDU,männlichmännlichmännlichmännlichmännlichmännlich,@fabiodemasi so ist estrotz dopingverstrickung...,0,0,2,0,0,0,0


In [27]:
polNdsCategorized = polNds
polNdsCategorized.to_csv("../../methods/representative-speech/politikerdaten/NIEDERSACHSEN_politikerKategorisiert.csv")

## NORDRHEIN-WESTFALEN

In [28]:
polNrw = polNrw.drop(columns='...1')

In [29]:
categPolNrw = {
    'covid' : ['corona', 'impfung', 'pandemie', 'covid', 'longcovid'],
    
    'energiekrise' : ['habeck', 'gas', 'energie', 'krise'],
    
    'politikInternational' : ['welt', 'china', 'usa', 'israel', 'staaten', 'europäische'],
    
    'politikNational' : ['ampel', 'scholz', 'spd', 'bundesregierung', 'fdp', 'regierung',
                         'union', 'bundeskanzler', 'opposition', 'kanzler', 'lambrecht', 'minister',
                         'grüne', 'bundestag', 'impfpflicht', 'debatte', 'antrag', 'mehrheit', 'gesetz',
                         'regierung', 'koalition', 'parlament', 'fraktion', 'koalitionsvertrag'],
    
    'klima' : ['zukunft', 'klimakrise', 'klimaschutz', 'ausbau', 'lützerath', 'erneuerbaren',
               'klima', 'energien', 'fossilen', 'energiewende'],
    
    'proteste' : ['freiheit', 'demokratie', 'leben', 'frauen', 'solidarität', 'menschenrechte', 'iran',
                  'hass', 'mut', 'rainbowflag', 'kampf', 'kämpfen'],
    
    'sozialesEntlastung' : ['inflation', 'sorgen', 'gasumlage', 'schuldenbremse', 'preise'],
    
    'ukraine' : ['ukraine', 'putin', 'russland', 'krieg', 'standwithukraine', 'putins', 'waffen', 'sanktionen',
                 'russischen', 'angriffskrieg', 'russische', 'nato', 'waffenlieferungen', 'russlands', 'angriff']
}

In [30]:
dfsPolNrw = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categPolNrw.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsPolNrw.append(words)

termsPolNrw = pd.concat(dfsPolNrw)
termsPolNrw

,category,term
0,covid,corona
1,covid,impfung
2,covid,pandemie
3,covid,covid
4,covid,longcovid
...,...,...
10,ukraine,russische
11,ukraine,nato
12,ukraine,waffenlieferungen
13,ukraine,russlands


In [31]:
# Take the 'term' column from our list of terms
termlistPolNrw = list(set(termsPolNrw.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistPolNrw)

matrix = vectorizer.fit_transform(polNrw.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [32]:
# Group the terms by category, then loop through each category
for category_name, rows in termsPolNrw.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polNrw[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covid 
Worte: ['corona', 'impfung', 'pandemie', 'covid', 'longcovid'] 

Topic: energiekrise 
Worte: ['habeck', 'gas', 'energie', 'krise'] 

Topic: klima 
Worte: ['zukunft', 'klimakrise', 'klimaschutz', 'ausbau', 'lützerath', 'erneuerbaren', 'klima', 'energien', 'fossilen', 'energiewende'] 

Topic: politikInternational 
Worte: ['welt', 'china', 'usa', 'israel', 'staaten', 'europäische'] 

Topic: politikNational 
Worte: ['ampel', 'scholz', 'spd', 'bundesregierung', 'fdp', 'regierung', 'union', 'bundeskanzler', 'opposition', 'kanzler', 'lambrecht', 'minister', 'grüne', 'bundestag', 'impfpflicht', 'debatte', 'antrag', 'mehrheit', 'gesetz', 'regierung', 'koalition', 'parlament', 'fraktion', 'koalitionsvertrag'] 

Topic: proteste 
Worte: ['freiheit', 'demokratie', 'leben', 'frauen', 'solidarität', 'menschenrechte', 'iran', 'hass', 'mut', 'rainbowflag', 'kampf', 'kämpfen'] 

Topic: sozialesEntlastung 
Worte: ['inflation', 'sorgen', 'gasumlage', 'schuldenbremse', 'preise'] 

Topic: ukra

In [41]:
polNrw.groupby('user').ukraine.sum().sort_values(ascending=False)

user
MatthiasHauer    910
n_roettgen       618
SaraNanni        531
BriHasselmann    392
SerapGueler      386
                ... 
busen_mdb          0
JensPeick          0
Renner_AfD         0
MTodtenhausen      0
wiebkeesdar1       0
Name: ukraine, Length: 111, dtype: int64

In [42]:
polNrw.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,geschlecht,text,covid,energiekrise,klima,politikInternational,politikNational,proteste,sozialesEntlastung,ukraine
user,,,,,,,,,,,,,,,,,,,,,
Achim_P,2022-10-20T21:39:57Z2022-10-09T16:46:44Z2022-1...,1.830457e+20,Herzlichen Glückwunsch @AliDogan1982!Friedrich...,233,252,1346,19,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/spdmilk/status/15832111157...,...,männlichmännlichmännlichmännlichmännlichmännli...,herzlichen glückwunsch @alidogan1982friedrich ...,3,4,4,11,31,7,2,11
Andreas_Rimkus,2022-10-31T13:54:15Z2022-10-31T08:24:02Z2022-1...,2.509759e+20,@Timon_Gremmels ...und natürlich Speicher ausb...,169,224,787,11,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/dena_news/status/155296641...,...,männlichmännlichmännlichmännlichmännlichmännli...,@timongremmels und natürlich speicher ausbauen...,1,7,8,9,24,8,4,6
AndrejHunko,2022-10-31T23:59:43Z2022-10-30T23:21:57Z2022-1...,1.709488e+20,#Bolsonaro will das Ergebnis weder anfechten n...,327,1468,3771,81,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/deepa_driver/status/157397...,...,männlichmännlichmännlichmännlichmännlichmännli...,bolsonaro will das ergebnis weder anfechten no...,3,0,1,8,27,3,0,24
AnjaKarliczek,2022-10-31T13:48:10Z2022-10-31T13:48:10Z2022-1...,1.420419e+20,2/2 Zudem subventionieren viele ausländische L...,428,314,1511,34,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/polenz_r/status/1571915695...,...,weiblichweiblichweiblichweiblichweiblichweibli...,22 zudem subventionieren viele ausländische lä...,3,9,4,3,18,5,7,6
AnneKoenig_,2022-10-31T15:25:31Z2022-10-31T12:41:47Z2022-1...,1.066852e+20,Gute Frage…Die SPD sieht sich ja gerne selbst ...,387,426,2354,38,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/BILD/status/15868413825131...,...,weiblichweiblichweiblichweiblichweiblichweibli...,gute frage…die spd sieht sich ja gerne selbst ...,0,12,4,2,35,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
svenlehmann,2022-10-31T15:26:48Z2022-10-31T07:43:09Z2022-1...,1.945635e+21,@UlrichSchneider Das glauben wir sofort! beam...,61422,43674,473468,9318,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/cdgham/status/158446072087...,...,männlichmännlichmännlichmännlichmännlichmännli...,@ulrichschneider das glauben wir sofort beami...,16,12,22,16,198,242,16,72
tj_tweets,2022-10-29T21:52:55Z2022-10-28T17:49:55Z2022-1...,1.018020e+21,"Liebe ist, wenn die eigenen Ehefrau vor dem Zu...",4121,4948,38158,691,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/JanDrebes/status/158204320...,...,männlichmännlichmännlichmännlichmännlichmännli...,liebe ist wenn die eigenen ehefrau vor dem zub...,11,17,10,21,124,9,15,27
ulle_schauws,2022-10-30T23:15:28Z2022-10-28T18:10:36Z2022-1...,2.316656e+20,#lula gewinnt! So eine wichtige Wahl - für #Br...,1583,798,7271,228,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/DLFNachrichten/status/1586...,...,weiblichweiblichweiblichweiblichweiblichweibli...,lula gewinnt so eine wichtige wahl für brasil...,1,2,1,3,22,37,0,5


In [43]:
polNrwCategorized = polNrw
polNrwCategorized.to_csv("../../methods/representative-speech/politikerdaten/NRW_politikerKategorisiert.csv")

## RHEINLAND-PFALZ

In [44]:
polRlp = polRlp.drop(columns='...1')

In [52]:
categPolRlp = {
    'ukraine' : ['ukraine', 'krieg', 'putin', 'russland', 'russischen', 'russische', 'putins'],
    
    'flutAhrtal' : ['flut', 'flutkatastrophe', 'ahr', 'ahrtal', 'ahrweiler', 'neuenahr', 'lewentz', 'hochwasser'],
    
    'politikNational' : ['grünen', 'spd', 'merz', 'gesundheitsminister', 'ampel', 'bundestag', 'impfpflicht',
                         'regierung', 'union', 'opposition', 'kanzler', 'rheinland-pfalz', 'staat', 'lambrecht',
                         'sondervermögen', 'scholz', 'spd', 'kanzler', 'bundestag', 'wirtschaft', 'habeck',
                         'bundeswehr', 'lauterbach'],
    
    'covid' : ['corona', 'eigenverantwortung', 'pandemie', 'gesundheitssystem', 'omikron'],
    
    'politikAfd' : ['afd', 'ideologie', 'unserlandzuerst', 'b0810', 'heisserherbst'],
    
    'sozialesEntlastung' : ['inflation', 'entlastung', 'progression', 'gasumlage', 'entlastungen',
                            'strom', 'entlasten']
}

In [53]:
dfsPolRlp = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categPolRlp.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsPolRlp.append(words)

termsPolRlp = pd.concat(dfsPolRlp)
termsPolRlp

,category,term
0,ukraine,ukraine
1,ukraine,krieg
2,ukraine,putin
3,ukraine,russland
4,ukraine,russischen
5,ukraine,russische
6,ukraine,putins
0,flutAhrtal,flut
1,flutAhrtal,flutkatastrophe
2,flutAhrtal,ahr


In [54]:
# Take the 'term' column from our list of terms
termlistPolRlp = list(set(termsPolRlp.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistPolRlp)

matrix = vectorizer.fit_transform(polRlp.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

In [55]:
# Group the terms by category, then loop through each category
for category_name, rows in termsPolRlp.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polRlp[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
#     digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covid 
Worte: ['corona', 'eigenverantwortung', 'pandemie', 'gesundheitssystem', 'omikron'] 

Topic: flutAhrtal 
Worte: ['flut', 'flutkatastrophe', 'ahr', 'ahrtal', 'ahrweiler', 'neuenahr', 'lewentz', 'hochwasser'] 

Topic: politikAfd 
Worte: ['afd', 'ideologie', 'unserlandzuerst', 'b0810', 'heisserherbst'] 

Topic: politikNational 
Worte: ['grünen', 'spd', 'merz', 'gesundheitsminister', 'ampel', 'bundestag', 'impfpflicht', 'regierung', 'union', 'opposition', 'kanzler', 'rheinland-pfalz', 'staat', 'lambrecht', 'sondervermögen', 'scholz', 'spd', 'kanzler', 'bundestag', 'wirtschaft', 'habeck', 'bundeswehr', 'lauterbach'] 

Topic: sozialesEntlastung 
Worte: ['inflation', 'entlastung', 'progression', 'gasumlage', 'entlastungen', 'strom', 'entlasten'] 

Topic: ukraine 
Worte: ['ukraine', 'krieg', 'putin', 'russland', 'russischen', 'russische', 'putins'] 



In [56]:
polRlp.groupby('user').flutAhrtal.sum().sort_values(ascending=False)

user
MechthildHeil      28
JuliaKloeckner     13
PSchnieder         10
JoSteiniger         8
S_Muenzenmaier      5
Erwin_Rueddel       5
Nicole_Hoechst      4
sandra_weeser       3
ManuelHoeferlin     3
MDiedenhofen        2
IsabelMackensen     2
Bina_Khan_          2
Wissing             1
jethoru             1
carina_konrad       0
thereallenzii       0
thomasgebhart       0
tobiaslindner       0
josefoster          0
crueffer            0
A_Gloeckner         0
baldy_daniel        0
andreasbleckmdb     0
ThomasHitschler     0
TabeaRoessner       0
Herbstbernd27       0
DrJoeWeingarten     0
BraFDP              0
verenahubertz       0
Name: flutAhrtal, dtype: int64

In [58]:
polRlp.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,einzug,partei,geschlecht,text,covid,flutAhrtal,politikAfd,politikNational,sozialesEntlastung,ukraine
user,,,,,,,,,,,,,,,,,,,,,
A_Gloeckner,2022-10-31T10:04:07Z2022-10-26T18:12:50Z2022-1...,5.642146e+19,"Diese Aussage ist wie so viele, wenig fundiert...",41,50,172,2,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/watch_union/status/1586445...,...,ErststimmeErststimmeErststimmeErststimmeErstst...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,weiblichweiblichweiblichweiblichweiblichweibli...,diese aussage ist wie so viele wenig fundiert ...,0,0,0,10,1,2
Bina_Khan_,2022-10-31T08:02:02Z2022-10-30T17:06:15Z2022-1...,2.766801e+20,Die Wahl von #Lula ist ein wichtiger Erfolg im...,1572,650,3519,97,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/rechte_gewalt/status/15859...,...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,B90/Die GrünenB90/Die GrünenB90/Die GrünenB90/...,weiblichweiblichweiblichweiblichweiblichweibli...,die wahl von lula ist ein wichtiger erfolg im ...,3,2,3,13,7,5
BraFDP,2022-10-30T19:02:07Z2022-10-17T19:21:27Z2022-1...,2.010563e+20,Das #Infomobil des Deutschen Bundestages zieht...,1953,320,2257,322,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/starkwatzinger/status/1582...,...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,FDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPF...,männlichmännlichmännlichmännlichmännlichmännli...,das infomobil des deutschen bundestages zieht ...,2,0,0,26,5,9
DrJoeWeingarten,2022-10-30T11:03:28Z2022-10-24T16:00:25Z2022-1...,1.938394e+20,Deutsch-quatarisches Parlamentariergespräch he...,4827,1025,6707,783,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AutorToto/status/157536626...,...,ErststimmeErststimmeErststimmeErststimmeErstst...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,deutsch-quatarisches parlamentariergespräch he...,1,0,2,26,3,12
Erwin_Rueddel,2022-10-30T17:28:52Z2022-10-29T14:00:00Z2022-1...,1.480022e+21,Das Bürgergeld setzt die absolut falschen Anre...,21014,17954,132542,1835,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/welt/status/15866286896174...,...,ErststimmeErststimmeErststimmeErststimmeErstst...,CDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUC...,männlichmännlichmännlichmännlichmännlichmännli...,das bürgergeld setzt die absolut falschen anre...,173,5,21,277,9,25
Herbstbernd27,2022-10-08T09:15:52Z2022-08-13T11:26:00Z,3.137090e+18,"@ZDFheute Und das ""zufällig"" wenn die AfD in B...",0,0,1,0,0,0.0,0,...,ZweitstimmeZweitstimme,AfDAfD,männlichmännlich,@zdfheute und das zufällig wenn die afd in ber...,0,0,1,0,0,0
IsabelMackensen,2022-10-09T20:03:02Z2022-09-26T17:31:07Z2022-0...,9.019031e+19,@Joergtaenzer @Alex_Schweitzer @derspiegel Auc...,98,70,146,8,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/Alex_Schweitzer/status/157...,...,ZweitstimmeZweitstimmeZweitstimmeZweitstimmeZw...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,weiblichweiblichweiblichweiblichweiblichweibli...,@joergtaenzer @alexschweitzer @derspiegel auch...,0,2,0,10,0,0
JoSteiniger,2022-10-31T09:39:37Z2022-10-29T16:53:38Z2022-1...,1.509914e+21,"Word!Erst mal übers Wochenende kleben lassen, ...",8768,12377,70387,2187,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/groehe/status/158689626356...,...,ErststimmeErststimmeErststimmeErststimmeErstst...,CDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUC...,männlichmännlichmännlichmännlichmännlichmännli...,worderst mal übers wochenende kleben lassen wü...,10,8,7,172,25,29
JuliaKloeckner,2022-10-31T13:34:08Z2022-10-30T20:58:50Z2022-1...,9.442594e+20,Als Teil der Bundesregierung kann man das ände...,30538,7757,63999,3152,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/DLFNachrichten/status/1585...,...,ZweitstimmeZw

In [59]:
polRlpCategorized = polRlp
polRlpCategorized.to_csv("../../methods/representative-speech/politikerdaten/RHEINLAND_politikerKategorisiert.csv")

## SAARLAND

In [60]:
polSaar = polSaar.drop(columns='...1')

In [67]:
categPolSaar = {
    'energie' : ['kohle', 'kernkraft', 'gas', 'strom', 'energiepreise', 'kernkraftwerke', 'fracking', 'erdgas', 'akw', 'abschalten', 'atomkraft', 'kohlekraftwerke', 'netz', 'gas', 'akw', 'energiekrise', 'energiepolitik', 'lng', 'tankrabatt', 'gaspreise', 'energiewende', 'versorgungssicherheit'],
    
    'ukraineKrieg' : ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia',
                      'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj',
                      'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato'],
    
    'ukraineBewaffnung' : ['waffen', 'waffenlieferungen', 'panzer', 'schützenpanzer', 'gepard',
                           'marder', 'munition', 'militärhilfe'],
    
    'ukraineReaktion' : ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine',
                         'mariupol', 'butscha', 'saporischschja', 'besatzer', 'aggressor'],
    
    'investitionen' : ['bundeswehr', 'milliarden', 'klimaschutz'],
    
    'flutAhrtal' : ['flut', 'flutkatastrophe', 'ahr', 'ahrtal', 'ahrweiler', 'neuenahr', 'lewentz', 'hochwasser'],
}

In [68]:
dfsPolSaar = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categPolSaar.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsPolSaar.append(words)

termsPolSaar = pd.concat(dfsPolSaar)
termsPolSaar

,category,term
0,energie,kohle
1,energie,kernkraft
2,energie,gas
3,energie,strom
4,energie,energiepreise
...,...,...
3,flutAhrtal,ahrtal
4,flutAhrtal,ahrweiler
5,flutAhrtal,neuenahr
6,flutAhrtal,lewentz


In [69]:
# Take the 'term' column from our list of terms
termlistPolSaar = list(set(termsPolSaar.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistPolSaar)

matrix = vectorizer.fit_transform(polSaar.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

#########################################################

# Group the terms by category, then loop through each category
for category_name, rows in termsPolSaar.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polSaar[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
    #digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: energie 
Worte: ['kohle', 'kernkraft', 'gas', 'strom', 'energiepreise', 'kernkraftwerke', 'fracking', 'erdgas', 'akw', 'abschalten', 'atomkraft', 'kohlekraftwerke', 'netz', 'gas', 'akw', 'energiekrise', 'energiepolitik', 'lng', 'tankrabatt', 'gaspreise', 'energiewende', 'versorgungssicherheit'] 

Topic: flutAhrtal 
Worte: ['flut', 'flutkatastrophe', 'ahr', 'ahrtal', 'ahrweiler', 'neuenahr', 'lewentz', 'hochwasser'] 

Topic: investitionen 
Worte: ['bundeswehr', 'milliarden', 'klimaschutz'] 

Topic: ukraineKrieg 
Worte: ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia', 'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj', 'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato'] 

Topic: ukraineReaktion 
Worte: ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine', 'mariupol', 'butscha', 'saporischschja', 'besatzer', 'aggressor'] 

Topic: ukraineWaffendiskurs 
Worte: [

In [72]:
polSaar.groupby('user').ukraineReaktion.sum().sort_values(ascending=False)

user
OlliLuksic         46
josephineortleb     1
CPetryMdB           0
ChrWirthMdB         0
EsraLimbacher       0
HeikoMaas           0
NadineSchoen        0
uhl_markus          0
Name: ukraineReaktion, dtype: int64

In [73]:
polSaar.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,partei,geschlecht,text,energie,flutAhrtal,investitionen,ukraine,ukraineKrieg,ukraineReaktion,ukraineWaffendiskurs
user,,,,,,,,,,,,,,,,,,,,,
CPetryMdB,2022-02-12T14:15:53Z2022-02-12T14:15:53Z2022-0...,7.450265e+18,Er muss sich entschuldigen oder Tobias Hans mu...,4,12,55,1,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,SPDSPDSPDSPDSPD,männlichmännlichmännlichmännlichmännlich,er muss sich entschuldigen oder tobias hans mu...,0,0,0,0,0,0,0
ChrWirthMdB,2022-10-28T15:21:21Z2022-10-27T14:08:25Z2022-1...,7.072232e+19,"Beruhigend, wie unser Staat funktioniert. Und ...",91,161,384,6,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/GtzFrmming/status/15855542...,...,AfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDAfDA...,männlichmännlichmännlichmännlichmännlichmännli...,beruhigend wie unser staat funktioniert und be...,0,0,1,1,2,0,1
EsraLimbacher,2022-10-27T09:53:42Z2022-10-26T10:32:44Z2022-1...,3.175471e+20,@matthiasfuchs_ Das würde mich sehr freuen sm...,1147,1404,15363,228,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/SRaktuell/status/156938740...,...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,@matthiasfuchs das würde mich sehr freuen smi...,10,0,6,6,8,0,2
HeikoMaas,2022-04-21T16:21:36Z2022-04-21T16:21:29Z2022-0...,2.692426e+19,Thorsten Bischoff hat mich viele Jahre auf mei...,790,416,4315,66,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/SAARTEXT/status/1517087797...,...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,männlichmännlichmännlichmännlichmännlichmännli...,thorsten bischoff hat mich viele jahre auf mei...,0,0,0,2,2,0,0
NadineSchoen,2022-10-22T18:54:11Z2022-10-20T21:48:23Z2022-1...,5.019514e+19,@erotte So? Was genau?@wenig_worte War ganz er...,70,71,536,6,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/flueke/status/158238528653...,...,CDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUCDUC...,weiblichweiblichweiblichweiblichweiblichweibli...,@erotte so was genau@wenigworte war ganz ernst...,1,0,0,0,0,0,0
OlliLuksic,2022-10-31T19:08:18Z2022-10-31T13:19:24Z2022-1...,5.259047e+21,@ChristophCanne @hahnflo Wir machen es hin The...,30583,22848,170409,2800,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/PreussenEl/status/15863465...,...,FDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPFDPF...,männlichmännlichmännlichmännlichmännlichmännli...,@christophcanne @hahnflo wir machen es hin the...,558,6,60,320,340,46,28
josephineortleb,2022-10-21T13:21:27Z2022-10-20T11:36:38Z2022-1...,1.375506e+20,"So sieht es übrigens aus, wenn die\nUnion sagt...",1604,746,7229,121,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/AnkeRehlinger/status/15826...,...,SPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDSPDS...,weiblichweiblichweiblichweiblichweiblichweibli...,so sieht es übrigens aus wenn die\nunion sagt ...,0,0,0,6,5,1,0
uhl_markus,2022-03-08T17:07:27Z2022-02-22T21:14:57Z2022-0...,4.493707e+18,@peteraltmaier Gute Besserung!@EsraLimbacher @...,2,3,4,0,0,0.0,0,...,CDUCDUCDU,männlichmännlichmännlich,@peteraltmaier gute besserung@esralimbacher @t...,0,0,0,0,0,0,0


In [100]:
polSaarCategorized = polSaar
polSaarCategorized.to_csv("../../methods/representative-speech/politikerdaten/SAARLAND_politikerKategorisiert.csv")

## SACHSEN

In [75]:
polSachsen = polSachsen.drop(columns='...1')

In [101]:
categPolSachsen = {
    'energieKrise' : ['energiekrise', 'energiepolitik', 'energie', 'gas', 'strom', 'öl', 'habeck',
                      'wirtschaftsminister', 'erneuerbare', 'erneuerbaren', 'energien', 'versorgungssicherheit',
                      'gasumlage', 'gaspreise', 'inflation',
                      'raffinerie', 'flüssiggas', 'lng', 'terminal', 'erdgas', 'erdöl', 'kohle',
                      'kohlekraftwerke', 'fracking', 'fossile', 'fossilen', 'pck', 'schwedt'],
    
    'energieAtom' : ['atomkraft', 'atomkraftwerk', 'atomkraftwerke', 'kernkraft', 'kernkraftwerk',
                     'kernkraftwerke', 'akw', 'akws', 'netz', 'nuclear'],
    
    'ukraineKrieg' : ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia',
                      'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj',
                      'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato'],
    
    'ukraineBewaffnung' : ['waffen', 'waffenlieferungen', 'panzer', 'schützenpanzer', 'gepard',
                           'marder', 'munition', 'militärhilfe'],
    
    'ukraineReaktion' : ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine',
                         'mariupol', 'butscha', 'saporischschja', 'besatzer', 'aggressor'],
    
    'sozialesWohnen' : ['wohnungsbau', 'mieter', 'bauen', 'mieterinnen', 'mieterin', 'miete',
                        'mietendeckel', 'mietenstopp', 'sozialwohnung', 'sozialwohnungen', 'armut'],
    
    'politikNational' : ['cdu', 'linke', 'afd', 'spd', 'partei', 'fdp', 'union', 'grüne', 'grünen',
                         'scholz', 'merz', 'koalitionsvertrag', 'bundestag', 'ampel', 'impfpflicht',
                         'bundesregierung', 'debatte', 'gesetzentwurf', 'mehrheit', 'ostdeutschland'],
    
    'sozialesEntlastung' : ['inflation', 'sorgen', 'preise', 'energiepreise', 'lebenshaltungskosten',
                            'lebenshaltung', 'entlasten', 'entlastung', 'entlastungen', 'entlastungspolitik',
                            'einkommen', 'heizkostenzuschuss', '9-euro-ticket', '9euroticket', 'progression',
                            'armut'],
    
    'flutAhrtal' : ['flut', 'flutkatastrophe', 'ahr', 'ahrtal', 'ahrweiler', 'neuenahr',
                    'lewentz', 'hochwasser'],
    
    'covidVirus' : ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'longcovid',
                    'omikron','delta', 'gesundheitsminister', 'lauterbach', 'gesundheitssystem'],
    
    'covidMaßnahmen' : ['maske', 'masken', 'maskenpflicht', 'corona-regeln', 'corona-lockerungen',
                        'eigenverantwortung', 'eigenverantwortlich'],
    
    'covidImpfung' : ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech',
                      'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern'],
    
    'covidNews' : ['rki', 'robert-koch-institut', 'inzidenz', 'neuinfektionen', 'corona-fälle',
                   'virologe', 'drosten']
}

In [102]:
dfsPolSachsen = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categPolSachsen.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsPolSachsen.append(words)

termsPolSachsen = pd.concat(dfsPolSachsen)
termsPolSachsen

,category,term
0,energieKrise,energiekrise
1,energieKrise,energiepolitik
2,energieKrise,energie
3,energieKrise,gas
4,energieKrise,strom
...,...,...
2,covidNews,inzidenz
3,covidNews,neuinfektionen
4,covidNews,corona-fälle
5,covidNews,virologe


In [103]:
# Take the 'term' column from our list of terms
termlistPolSachsen = list(set(termsPolSachsen.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistPolSachsen)

matrix = vectorizer.fit_transform(polSachsen.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

#########################################################

# Group the terms by category, then loop through each category
for category_name, rows in termsPolSachsen.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polSachsen[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
    #digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern'] 

Topic: covidMaßnahmen 
Worte: ['maske', 'masken', 'maskenpflicht', 'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich'] 

Topic: covidNews 
Worte: ['rki', 'robert-koch-institut', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten'] 

Topic: covidVirus 
Worte: ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'longcovid', 'omikron', 'delta', 'gesundheitsminister', 'lauterbach', 'gesundheitssystem'] 

Topic: energieAtom 
Worte: ['atomkraft', 'atomkraftwerk', 'atomkraftwerke', 'kernkraft', 'kernkraftwerk', 'kernkraftwerke', 'akw', 'akws', 'netz', 'nuclear'] 

Topic: energieKrise 
Worte: ['energiekrise', 'energiepolitik', 'energie', 'gas', 'strom', 'öl', 'habeck', 'wirtschaftsminister', 'erneuerbare', 'erneuerbaren', 'energien', 'versorgungssicherheit', 'gasumlage', '

In [107]:
polSachsen.groupby('user').covidMaßnahmen.sum().sort_values(ascending=False)

user
torstenherbst      12
BHerrmann_         10
philipphartewig     8
katjakipping        4
CarenLay            4
MuellerChemnitz     3
LINKEPELLI          2
wanderwitz          2
MMoosdorf           0
kassem_ts           0
CarstenKoerber      0
theliberalfrank     0
spellerberg_m       0
rasha_nasr_         0
HarzerUlrike        0
HilseMdb            0
funke_fabian        0
MarkusReichelDD     0
edgar_naujok        0
YvonneMagwas        0
Tino_Chrupalla      0
HolgerMannLE        0
Kthrn_Mchl_SPD      0
Mike_Moncsek        0
PaulaPiechotta      0
Name: covidMaßnahmen, dtype: int64

In [108]:
polSachsen.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,sozialesEntlastung,sozialesWohnen,ukraineBewaffnung,ukraineKrieg,ukraineReaktion,flutAhrtal,covidImpfung,covidMaßnahmen,covidNews,covidVirus
user,,,,,,,,,,,,,,,,,,,,,
BHerrmann_,2022-10-31T20:24:40Z2022-10-31T20:22:55Z2022-1...,5.550824e+21,Wir haben hier im Land und gerade auch in Sach...,4252,7410,24030,336,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BHerrmann_/status/15836679...,...,122,24,0,48,2,2,2,10,0,4
CarenLay,2022-10-31T13:12:46Z2022-10-30T10:59:00Z2022-1...,2.244476e+21,#Vorkaufsrecht: Das Berliner Verwaltungsgerich...,7674,12286,60853,1292,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/BaharAslan_/status/1586646...,...,108,356,8,92,20,0,6,4,0,26
CarstenKoerber,2022-09-29T11:44:20Z2022-09-28T18:59:35Z2022-0...,2.127991e+19,Schloss #Forderglauchau und #Alumnat des Kreuz...,29,33,196,1,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,1,0,1,6,1,0,1,0,0,0
HarzerUlrike,2022-10-27T14:31:24Z2022-10-27T10:03:38Z2022-1...,3.810684e+20,#SteuerschätzungNach Erfahrungen mit #Hamburge...,246,488,1382,26,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/Meyer_FDP/status/158562554...,...,38,0,0,20,2,0,2,0,0,4
HilseMdb,2022-10-29T12:02:43Z2022-10-28T11:27:27Z2022-0...,1.858491e+19,Geliefert wie bestellt (gewählt)! https://t.co...,42,84,224,4,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,0,0,0,0,0,0,0,0,0,0
HolgerMannLE,2022-10-25T14:11:41Z2022-10-25T10:54:07Z2022-1...,8.627485e+20,Lesenswert zum #Buergergeld. \n#AusRespekt \n@...,800,1388,5828,134,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/sozi_simon/status/15845359...,...,26,0,6,82,16,0,4,0,0,6
Kthrn_Mchl_SPD,2022-10-28T11:09:46Z2022-10-06T13:01:06Z2022-0...,3.783100e+19,"Komisch, dass diejenigen, die gegen das #Spree...",9,57,195,6,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/reDDakteur/status/15859059...,...,2,0,0,1,1,0,1,0,0,0
LINKEPELLI,2022-10-31T18:11:30Z2022-10-30T21:05:04Z2022-1...,5.561334e+20,#Genugistgenug! Gruselpolitik beenden. Unter d...,2445,2875,14397,348,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/DIELINKELeipzig/status/158...,...,22,7,5,17,2,0,4,2,0,5
MMoosdorf,2022-10-10T18:30:51Z2022-10-10T18:27:02Z2022-1...,3.427111e+19,@n_roettgen Ihre politische Kurzsichtigkeit is...,2,1,13,0,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,0,...,0,0,1,1,1,0,0,0,0,0


In [109]:
polSachsenCategorized = polSachsen
polSachsenCategorized.to_csv("../../methods/representative-speech/politikerdaten/SACHSEN_politikerKategorisiert.csv")

## SACHSEN-ANHALT

In [97]:
polSachsenAnhalt = polSachsenAnhalt.drop(columns='...1')

In [111]:
categPolSachAn = {
    'energieKrise' : ['energiekrise', 'energiepolitik', 'energie', 'gas', 'strom', 'öl', 'habeck',
                      'wirtschaftsminister', 'erneuerbare', 'erneuerbaren', 'energien', 'versorgungssicherheit',
                      'gasumlage', 'gaspreise', 'inflation',
                      'raffinerie', 'flüssiggas', 'lng', 'terminal', 'erdgas', 'erdöl', 'kohle',
                      'kohlekraftwerke', 'fracking', 'fossile', 'fossilen', 'pck', 'schwedt'],
    
    'energieAtom' : ['atomkraft', 'atomkraftwerk', 'atomkraftwerke', 'kernkraft', 'kernkraftwerk',
                     'kernkraftwerke', 'akw', 'akws', 'netz', 'nuclear'],
    
    'ukraineKrieg' : ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia',
                      'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj',
                      'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato'],
    
    'ukraineBewaffnung' : ['waffen', 'waffenlieferungen', 'panzer', 'schützenpanzer', 'gepard',
                           'marder', 'fuchs', 'munition', 'militärhilfe', 'panzerhaubitzen'],
    
    'ukraineReaktion' : ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine',
                         'mariupol', 'butscha', 'saporischschja', 'cherson', 'besatzer', 'aggressor'],
    
    'sozialesWohnen' : ['wohnungsbau', 'mieter', 'bauen', 'mieterinnen', 'mieterin', 'miete',
                        'mietendeckel', 'mietenstopp', 'sozialwohnung', 'sozialwohnungen', 'armut'],
    
    'sozialesEntlastung' : ['inflation', 'sorgen', 'preise', 'energiepreise', 'lebenshaltungskosten',
                            'lebenshaltung', 'entlasten', 'entlastung', 'entlastungen', 'entlastungspolitik',
                            'einkommen', 'heizkostenzuschuss', '9-euro-ticket', '9euroticket', 'progression',
                            'armut'],
    
    'flutAhrtal' : ['flut', 'flutkatastrophe', 'ahr', 'ahrtal', 'ahrweiler', 'neuenahr',
                    'lewentz', 'hochwasser'],
    
    'covidVirus' : ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'longcovid',
                    'omikron','delta', 'gesundheitsminister', 'lauterbach', 'gesundheitssystem'],
    
    'covidMaßnahmen' : ['maske', 'masken', 'maskenpflicht', 'corona-regeln', 'corona-lockerungen',
                        'eigenverantwortung', 'eigenverantwortlich'],
    
    'covidImpfung' : ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech',
                      'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern'],
    
    'covidNews' : ['rki', 'robert-koch-institut', 'inzidenz', 'neuinfektionen', 'corona-fälle',
                   'virologe', 'drosten'],
    
    'verteidigungspolitik' : ['zeitenwende', 'modernisieren', 'modernisierung', 'verteidigungsministerin',
                              'verteidigungsminister', 'verteidigungsministerium', 'verteidigungspolitik',
                              'sondervermögen'],
    
    'politikNational' : ['ampel', 'impfpflicht', 'bundesregierung', 'regierung', 'bundestag', 'afd',
                         'lauterbach', 'grüne', 'spd']
}

In [112]:
dfsPolSachAn = []
# stemmer=Stemmer.Stemmer('de')

for key,values in categPolSachAn.items():
    words=pd.DataFrame({'category': key, 'term': values}) ## stemmer löschen, wenn ganze wörter gewollt sind
    dfsPolSachAn.append(words)

termsPolSachAn = pd.concat(dfsPolSachAn)
termsPolSachAn

,category,term
0,energieKrise,energiekrise
1,energieKrise,energiepolitik
2,energieKrise,energie
3,energieKrise,gas
4,energieKrise,strom
...,...,...
4,politikNational,bundestag
5,politikNational,afd
6,politikNational,lauterbach
7,politikNational,grüne


In [113]:
# Take the 'term' column from our list of terms
termlistPolSachAn = list(set(termsPolSachAn.term))

# binary=True only does 0/1
# vocabulary= is the list of words we're interested in tracking
vectorizer = TfidfVectorizer(binary=False, vocabulary = termlistPolSachAn)

matrix = vectorizer.fit_transform(polSachsenAnhalt.text.astype('U'), )

words_df = pd.DataFrame(matrix.toarray(), columns = vectorizer.get_feature_names_out())

#########################################################

# Group the terms by category, then loop through each category
for category_name, rows in termsPolSachAn.groupby('category'):
    # Convert the terms for that category into a simple list
    # for example, ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Topic: {category_name} \nWorte: {terms} \n")

    # words_df[terms]  -> die einzelnen bestandteile der categories (also die wörter der topics)
    # .any(axis=1) -> prüfen, ob irgendeines von den wörtern != 0
    # .astype(int) macht aus True bzw. False einen binären atomic vector
    # df[category_name] -> wird dann zu dem jeweiligen df als variable "category_name" zugewiesen
    polSachsenAnhalt[category_name] = words_df[terms].any(axis=1).astype(int)
    #politiker[category_name] = politiker[category_name].fillna(0)
    #politiker[category_name] = politiker[category_name].astype(int)
    #digitale[category_name] = digitale[category_name].astype('Int64', errors='ignore')

Topic: covidImpfung 
Worte: ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech', 'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern'] 

Topic: covidMaßnahmen 
Worte: ['maske', 'masken', 'maskenpflicht', 'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich'] 

Topic: covidNews 
Worte: ['rki', 'robert-koch-institut', 'inzidenz', 'neuinfektionen', 'corona-fälle', 'virologe', 'drosten'] 

Topic: covidVirus 
Worte: ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'longcovid', 'omikron', 'delta', 'gesundheitsminister', 'lauterbach', 'gesundheitssystem'] 

Topic: energieAtom 
Worte: ['atomkraft', 'atomkraftwerk', 'atomkraftwerke', 'kernkraft', 'kernkraftwerk', 'kernkraftwerke', 'akw', 'akws', 'netz', 'nuclear'] 

Topic: energieKrise 
Worte: ['energiekrise', 'energiepolitik', 'energie', 'gas', 'strom', 'öl', 'habeck', 'wirtschaftsminister', 'erneuerbare', 'erneuerbaren', 'energien', 'versorgungssicherheit', 'gasumlage', '

In [119]:
polSachsenAnhalt.groupby('user').covidMaßnahmen.sum().sort_values(ascending=False)

user
M_Reichardt_AfD    18
TinoSorge          11
mueller_sepp        4
jws_afd             2
IngoBodtke          1
KarambaDiaby        1
MarcusFaber         1
Petra_Sitte_MdB     1
jankortemdb         1
robert_farle        1
HeikeBrehmerMdB     0
KayUweZiegler71     0
SteffiLemke         0
Name: covidMaßnahmen, dtype: int64

In [120]:
polSachsenAnhalt.groupby('user', ).sum()

,dateTime,tweetID,content,replies,retweets,likes,quotes,media,isRetweeted,isQuoted,...,energieAtom,energieKrise,flutAhrtal,politikNational,sozialesEntlastung,sozialesWohnen,ukraineBewaffnung,ukraineKrieg,ukraineReaktion,verteidigungspolitik
user,,,,,,,,,,,,,,,,,,,,,
HeikeBrehmerMdB,2022-09-29T16:23:19Z2022-08-17T08:47:09Z2022-0...,1.678336e+19,Der Bund fördert das Schloss #Wernigerode mit ...,15,49,131,5,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/sachsenanhalt/status/14914...,...,0,0,0,1,0,0,0,0,0,0
IngoBodtke,2022-10-01T08:03:17Z2022-09-22T15:47:47Z2022-0...,4.808575e+20,Eine Verzehnfachung der #Energiekosten zerstör...,2738,518,4073,224,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/c_lindner/status/151786947...,...,1,0,0,29,14,0,3,22,4,3
KarambaDiaby,2022-10-31T10:54:28Z2022-10-09T11:25:27Z2022-0...,9.124355e+19,Glückwunsch und Parabéns @lulaoficial zur Präs...,2330,1407,24140,418,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/Bundeskanzler/status/15754...,...,0,1,0,6,2,1,0,5,1,0
KayUweZiegler71,2022-04-29T19:05:46Z2022-04-28T07:25:31Z2022-0...,7.585173e+18,"@SHomburg @MrsMertes ""Die Impfstoffe entfalten...",3,5,2,0,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,0,...,0,0,0,3,0,0,0,0,0,0
M_Reichardt_AfD,2022-10-30T18:37:45Z2022-10-30T18:11:32Z2022-1...,1.882518e+21,"@rawnezz_picture @Die_Gruenen Ja, die Firma is...",10287,18130,71225,990,[Video(thumbnailUrl='https://pbs.twimg.com/ext...,0.0,https://twitter.com/spdbt/status/1585937861232...,...,9,72,6,263,45,13,30,166,11,2
MarcusFaber,2022-10-31T19:50:37Z2022-10-31T10:36:02Z2022-1...,2.721768e+21,#Charkiw brennt weil #Putin es bombardiert. Mo...,32836,35740,288006,3314,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/FDP_LSA/status/15862556271...,...,5,19,0,87,16,3,304,844,232,271
Petra_Sitte_MdB,2022-10-31T14:46:47Z2022-10-23T08:24:07Z2022-1...,3.858403e+20,Gruselig sind derzeit auch die Preise - der Sc...,378,941,3370,81,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/pascalmeiser/status/158280...,...,3,6,0,48,12,4,0,12,2,3
SteffiLemke,2022-10-24T10:53:01Z2022-10-17T17:30:53Z2022-1...,2.378032e+20,Der Rat der EU Umweltminister hat heute das ei...,7638,2968,24036,844,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/IlkoKowalczuk/status/15764...,...,4,8,0,4,4,0,0,2,0,0
TinoSorge,2022-10-31T20:22:01Z2022-10-25T12:14:38Z2022-1...,3.742831e+20,Wechsel zwischen Wirtschaft &amp; Politik &amp...,8724,9576,59405,859,[Photo(previewUrl='https://pbs.twimg.com/media...,0.0,https://twitter.com/welt/status/15845323189619...,...,2,6,0,97,5,0,1,7,1,1


In [121]:
polSachAnCategorized = polSachsenAnhalt
polSachAnCategorized.to_csv("../../methods/representative-speech/politikerdaten/SACHSENANHALT_politikerKategorisiert.csv")

## SCHLESWIG-HOLSTEIN

In [122]:
polSchleswig = polSchleswig.drop(columns='...1')

In [123]:
categPolSchleswig = {
    'energieKrise' : ['energiekrise', 'energiepolitik', 'energie', 'gas', 'strom', 'öl', 'habeck',
                      'wirtschaftsminister', 'erneuerbare', 'erneuerbaren', 'erneuerbareenergien',
                      'energien', 'versorgungssicherheit', 'gasumlage', 'gaspreise', 'inflation',
                      'raffinerie', 'flüssiggas', 'lng', 'terminal', 'erdgas', 'erdöl', 'kohle',
                      'kohlekraftwerke', 'fracking', 'fossile', 'fossilen', 'pck', 'schwedt'],
    
    'energieAtom' : ['atomkraft', 'atomkraftwerk', 'atomkraftwerke', 'kernkraft', 'kernkraftwerk',
                     'kernkraftwerke', 'akw', 'akws', 'netz', 'nuclear'],
    
    'ukraineKrieg' : ['ukraine', 'ukrainer', 'ukrainische', 'ukrainischen', 'russland', 'russen', 'russia',
                      'russische', 'russischen', 'putin', 'putins', 'selenskyi', 'selensky', 'selenskyj',
                      'zelenskyy', 'selenskyis', 'selenskyjs', 'selenskys', 'zelenskyys', 'nato'],
    
    'ukraineBewaffnung' : ['waffen', 'waffenlieferungen', 'panzer', 'schützenpanzer', 'gepard',
                           'marder', 'fuchs', 'munition', 'militärhilfe', 'panzerhaubitzen'],
    
    'ukraineReaktion' : ['sanktionen', 'angriffskrieg', 'invasion', 'kriegsverbrechen', 'standwithukraine',
                         'mariupol', 'butscha', 'saporischschja', 'cherson', 'charkiw', 'besatzer', 'aggressor'],
    
    'sozialesWohnen' : ['wohnungsbau', 'mieter', 'bauen', 'mieterinnen', 'mieterin', 'miete',
                        'mietendeckel', 'mietenstopp', 'sozialwohnung', 'sozialwohnungen', 'armut'],
    
    'sozialesEntlastung' : ['inflation', 'sorgen', 'preise', 'energiepreise', 'lebenshaltungskosten',
                            'lebenshaltung', 'entlasten', 'entlastung', 'entlastungen', 'entlastungspolitik',
                            'einkommen', 'heizkostenzuschuss', '9-euro-ticket', '9euroticket', 'progression',
                            'armut'],
    
    'flutAhrtal' : ['flut', 'flutkatastrophe', 'ahr', 'ahrtal', 'ahrweiler', 'neuenahr',
                    'lewentz', 'hochwasser'],
    
    'covidVirus' : ['pandemie', 'coronavirus', 'covid19', 'covid', 'virus', 'corona', 'longcovid',
                    'omikron','delta', 'gesundheitsminister', 'lauterbach', 'gesundheitssystem'],
    
    'covidMaßnahmen' : ['maske', 'masken', 'maskenpflicht', 'ausgangssperre', 'ausgangssperren',
                        'corona-regeln', 'corona-lockerungen', 'eigenverantwortung', 'eigenverantwortlich'],
    
    'covidImpfung' : ['impfstoff', 'impfpflicht', 'impfen', 'impfung', 'geimpft', 'biontech',
                      'pfizer', 'astrazeneca', 'moderna', 'booster', 'boostern'],
    
    'covidNews' : ['rki', 'robert-koch-institut', 'inzidenz', 'neuinfektionen', 'corona-fälle',
                   'virologe', 'drosten'],
    
    'verteidigungspolitik' : ['zeitenwende', 'modernisieren', 'modernisierung', 'verteidigungsministerin',
                              'verteidigungsminister', 'verteidigungsministerium', 'verteidigungspolitik',
                              'sondervermögen'],
    
    'politikNational' : [],
    
    'politikInternational' : []
}